In [4]:
# 📌 Celda A - Reinicio - Configuración inicial y verificación de entorno
import sys
import platform
import pkg_resources

print(f"Python {platform.python_version()}")

# Versión de librerías clave
for pkg in ["llama-index-core", "llama-index-vector-stores-chroma", 
            "llama-index-embeddings-huggingface", "chromadb"]:
    try:
        version = pkg_resources.get_distribution(pkg).version
        print(f"{pkg}: {version}")
    except pkg_resources.DistributionNotFound:
        print(f"{pkg}: ❌ No instalado")


Python 3.10.18
llama-index-core: 0.12.20
llama-index-vector-stores-chroma: 0.4.2
llama-index-embeddings-huggingface: 0.5.2
chromadb: 0.5.23


In [2]:
# 📌 Celda B - Reinicio -  Restaurar almacenamiento existente
# 📌 Celda B - Reinicio - Restaurar almacenamiento existente
# 📌 Restaurar índice usando siempre el último registrado en index_store.json

import os, json
import chromadb
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

# ⚠️ Rutas fijas de tu proyecto
BASE_DIR = r"C:\batch001"
STORAGE_DIR = os.path.join(BASE_DIR, "store_test_yaml2")
ONTOLOGY_DIR = os.path.join(BASE_DIR, "ontology")  # ✅ agregado

# Cliente y colección de Chroma
chroma_client = chromadb.PersistentClient(path=os.path.join(STORAGE_DIR, "chroma"))
vector_store = ChromaVectorStore(
    chroma_collection=chroma_client.get_or_create_collection("riesgos_collection")
)

# Restaurar StorageContext
storage_context = StorageContext.from_defaults(
    vector_store=vector_store,
    persist_dir=STORAGE_DIR
)

# Embeddings
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

# 📌 Leer index_store.json y tomar el último índice
index_store_path = os.path.join(STORAGE_DIR, "index_store.json")
with open(index_store_path, "r", encoding="utf-8") as f:
    index_store_data = json.load(f)

index_ids = list(index_store_data["index_store/data"].keys())
print("📂 Índices encontrados en index_store.json:", index_ids)

if not index_ids:
    raise ValueError("❌ No se encontraron índices en index_store.json")

# Seleccionar siempre el último índice
last_index_id = index_ids[-1]
print(f"📌 Usando el último índice: {last_index_id}")

# Restaurar ese índice
index = load_index_from_storage(
    storage_context=storage_context,
    index_id=last_index_id,
    embed_model=embed_model
)

print(f"✅ Contexto e índice restaurados desde: {STORAGE_DIR}")




C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


📂 Índices encontrados en index_store.json: ['e2561835-cc05-4718-bf00-47fbe61d9edb', 'ac01d0f3-edd6-4fd6-882f-2ab09e63280e', '89c9bad8-3bf1-4964-b794-ffb5ae035f56', 'f9ce12f8-809c-4ca6-ae7b-217c88fb6ff9', '81119653-c271-40b5-951c-16b0064f4fb6', '4fffbf09-21e4-4fd2-8a2a-3c74f3bbff6e', 'af876b97-49fc-4456-931e-58b5db2712e8', 'cec01859-1075-457e-a7aa-92c0b0d6a2b3', 'b32d1fab-0e6a-40c6-86dc-5ac0cea0f550', '5addf3ca-93db-4df3-93de-9131b281b9e1', 'f0848b98-2b91-4420-97fc-0a3576e3a23f']
📌 Usando el último índice: f0848b98-2b91-4420-97fc-0a3576e3a23f
✅ Contexto e índice restaurados desde: C:\batch001\store_test_yaml2


In [5]:
# 📌 Celda C — Verificación de sincronización después de restaurar el índice

# 1️⃣ Total de nodos en docstore
docstore_ids = set(index.docstore.docs.keys())
print(f"📚 Total de nodos en docstore: {len(docstore_ids)}")

# 2️⃣ Total de embeddings en Chroma
chroma_data = vector_store._collection.get(limit=999999)  # obtener todos
chroma_ids = set(chroma_data["ids"])
print(f"🟦 Total de embeddings en Chroma: {len(chroma_ids)}")

# 3️⃣ Comparación de IDs
faltan_en_chroma = docstore_ids - chroma_ids
faltan_en_docstore = chroma_ids - docstore_ids

if not faltan_en_chroma and not faltan_en_docstore:
    print("✅ Docstore y Chroma están sincronizados (mismos IDs).")
else:
    print("⚠️ Desincronización detectada:")
    if faltan_en_chroma:
        print(f"   → IDs en docstore pero no en Chroma: {faltan_en_chroma}")
    if faltan_en_docstore:
        print(f"   → IDs en Chroma pero no en docstore: {faltan_en_docstore}")

# 4️⃣ Mostrar un ejemplo de nodo restaurado
if docstore_ids:
    ejemplo_id = list(docstore_ids)[0]
    ejemplo_node = index.docstore.docs[ejemplo_id]
    print("\n🔍 Ejemplo de nodo restaurado:")
    print("ID:", ejemplo_node.node_id)
    print("Texto:", ejemplo_node.text[:150], "...")
    print("Metadatos:", ejemplo_node.metadata)


📚 Total de nodos en docstore: 533
🟦 Total de embeddings en Chroma: 533
✅ Docstore y Chroma están sincronizados (mismos IDs).

🔍 Ejemplo de nodo restaurado:
ID: FE_128
Texto: Factor de Exposición: Materiales Mal Apilados. Almacenar cables, repuestos o herramientas sin racks adecuados o sin respetar límites de carga puede pr ...
Metadatos: {'tareas_asociadas_ids': 'TME_TA_013', 'tareas_asociadas_nombres': 'Acceso y manipulación de materiales en estanterías elevadas del almacén', 'tiene_tareas_asociadas': True, 'riesgos_asociados_ids': 'R01, R02', 'riesgos_asociados_nombres': 'Caída desde Altura, Caída de Objetos', 'tiene_riesgos_asociados': True, 'roles_asociados_ids': 'TME_ROL_07', 'roles_asociados_nombres': 'Encargado de Bodega de Repuestos Eléctricos', 'tiene_roles_asociados': True, 'areas_asociadas_ids': 'TME', 'areas_asociadas_nombres': 'Taller de Mantenimiento Eléctrico', 'tiene_areas_asociadas': True}


In [1]:
# 📌 Celda 1 — Verificar versión de Python y librerías clave
!python --version

!pip show llama-index-core
!pip show llama-index-vector-stores-chroma
!pip show llama-index-embeddings-huggingface
!pip show chromadb
!pip show sentence-transformers
!pip show transformers
!pip show tokenizers
!pip show numpy


Python 3.10.18
Name: llama-index-core
Version: 0.12.20
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: c:\users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages
Requires: aiohttp, dataclasses-json, deprecated, dirtyjson, filetype, fsspec, httpx, nest-asyncio, networkx, nltk, numpy, pillow, pydantic, PyYAML, requests, SQLAlchemy, tenacity, tiktoken, tqdm, typing-extensions, typing-inspect, wrapt
Required-by: llama-index-embeddings-huggingface, llama-index-vector-stores-chroma
Name: llama-index-vector-stores-chroma
Version: 0.4.2
Summary: llama-index vector_stores chroma integration
Home-page: 
Author: 
Author-email: Your Name <you@example.com>
License-Expression: MIT
Location: c:\users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages
Requires: chromadb, llama-index-core
Required-by: 
Name: llama-index-embeddings-huggingface
Version: 0.5.2
Summary: llama-index embeddings 

In [1]:
# 📌 Celda 2 — Configuración de rutas locales para la persistencia del índice
import os

# 📂 Ruta raíz de trabajo local
BASE_DIR = r"C:\\batch001"

# 📂 Carpeta de almacenamiento para este test específico
STORAGE_DIR = os.path.join(BASE_DIR, "store_test_yaml2")

# 📂 Carpeta donde están los YAML de ontología
ONTOLOGY_DIR = os.path.join(BASE_DIR, "ontology")

# Crear carpeta de almacenamiento si no existe
os.makedirs(STORAGE_DIR, exist_ok=True)

print(f"📂 Carpeta base: {BASE_DIR}")
print(f"📂 Carpeta almacenamiento: {STORAGE_DIR}")
print(f"📂 Carpeta ontología: {ONTOLOGY_DIR}")


📂 Carpeta base: C:\\batch001
📂 Carpeta almacenamiento: C:\\batch001\store_test_yaml2
📂 Carpeta ontología: C:\\batch001\ontology


In [2]:
# 📌 Celda 3 — Inicialización de Chroma y StorageContext sin persist_dir inicial
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext

# Inicializar cliente de Chroma persistente
chroma_client = chromadb.PersistentClient(path=os.path.join(STORAGE_DIR, "chroma"))
chroma_collection = chroma_client.get_or_create_collection("riesgos_collection")

# Crear VectorStore
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# Crear StorageContext (sin persist_dir para evitar lectura de archivos inexistentes)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

print("✅ StorageContext creado correctamente (sin persist_dir inicial).")



✅ StorageContext creado correctamente (sin persist_dir inicial).


In [3]:
# 📌 Celda 4 — Cargar modelo de embeddings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Modelo recomendado para semántica multilingüe (BAAI/bge-m3) o usar MiniLM si quieres más velocidad
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

print("✅ Modelo de embeddings cargado correctamente.")


C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\eduar\anaconda3\envs\vbd_01245_local\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


✅ Modelo de embeddings cargado correctamente.


In [4]:
# 📌 Celda 5 — Preparar nodos desde el YAML de riesgos
# 📌 Celda 5 — R1′ Crear nodos enriquecidos de Riesgos

import yaml
from llama_index.core.schema import TextNode

# === Función utilitaria para cargar YAML ===
def load_yaml(path):
    with open(path, "r", encoding="utf-8") as f:
        return yaml.safe_load(f)

# === Cargar catálogos de referencia ===
mapa_causas = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "03_catalogo_causas_v4.yaml"))}
mapa_controles = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml"))}
mapa_consecuencias = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "04_catalogo_consecuencias_v6.yaml"))}
mapa_factores = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "05_catalogo_factores_degradacion_v3.yaml"))}
mapa_barreras = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "06_catalogo_barreras_recuperacion_v3.yaml"))}
mapa_exposicion = {item["id"]: item["nombre"] for item in load_yaml(os.path.join(ONTOLOGY_DIR, "11_catalogo_factores_exposicion_v3.yaml"))}

# === Cargar catálogo de riesgos ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
riesgos = load_yaml(yaml_riesgos)

# Campos base
campos_permitidos_riesgos = {
    "aplicabilidad", "tags", "clasificacion", "registro_aplicable_a", "area"
}

# Relaciones a convertir en flags
relaciones_flags_riesgos = {
    "causas_asociadas": "tiene_causas_asociadas",
    "controles_preventivos_asociados": "tiene_controles_preventivos",
    "controles_criticos_asociados": "tiene_controles_criticos",
    "controles_mitigacion_asociados": "tiene_controles_mitigacion",
    "factores_degradacion_asociados": "tiene_factores_degradacion",
    "consecuencias_asociadas": "tiene_consecuencias_asociadas",
    "barreras_recuperacion_asociadas": "tiene_barreras_recuperacion",
    "factores_exposicion_relacionados": "tiene_factores_exposicion"
}

nodes_riesgos = []
for item in riesgos:
    content = f"Riesgo: {item.get('nombre', '')}. {item.get('descripcion', '')}"
    metadata = {}

    # Procesar campos base
    for k in campos_permitidos_riesgos:
        if k == "tags":
            tags = item.get("tags", [])
            metadata[k] = ", ".join(tags) if isinstance(tags, list) else str(tags)
        elif k == "registro_aplicable_a":
            lista = item.get(k, [])
            metadata[k] = ", ".join(lista) if isinstance(lista, list) else str(lista)
        else:
            metadata[k] = item.get(k)

    # Procesar relaciones
    for campo_yaml, campo_flag in relaciones_flags_riesgos.items():
        lista_ids = item.get(campo_yaml, [])
        metadata[campo_flag] = bool(lista_ids)

        if campo_yaml == "causas_asociadas" and lista_ids:
            metadata["causas_asociadas_ids"] = ", ".join(lista_ids)
            metadata["causas_asociadas_nombres"] = ", ".join([mapa_causas.get(cid, "Desconocida") for cid in lista_ids])

        elif campo_yaml in ["controles_preventivos_asociados", "controles_criticos_asociados", "controles_mitigacion_asociados"] and lista_ids:
            metadata[f"{campo_yaml}_ids"] = ", ".join(lista_ids)
            metadata[f"{campo_yaml}_nombres"] = ", ".join([mapa_controles.get(cid, "Desconocido") for cid in lista_ids])

        elif campo_yaml == "factores_degradacion_asociados" and lista_ids:
            metadata["factores_degradacion_ids"] = ", ".join(lista_ids)
            metadata["factores_degradacion_nombres"] = ", ".join([mapa_factores.get(fid, "Desconocido") for fid in lista_ids])

        elif campo_yaml == "consecuencias_asociadas" and lista_ids:
            metadata["consecuencias_asociadas_ids"] = ", ".join(lista_ids)
            metadata["consecuencias_asociadas_nombres"] = ", ".join([mapa_consecuencias.get(cid, "Desconocida") for cid in lista_ids])

        elif campo_yaml == "barreras_recuperacion_asociadas" and lista_ids:
            metadata["barreras_recuperacion_ids"] = ", ".join(lista_ids)
            metadata["barreras_recuperacion_nombres"] = ", ".join([mapa_barreras.get(bid, "Desconocida") for bid in lista_ids])

        elif campo_yaml == "factores_exposicion_relacionados" and lista_ids:
            metadata["factores_exposicion_ids"] = ", ".join(lista_ids)
            metadata["factores_exposicion_nombres"] = ", ".join([mapa_exposicion.get(fid, "Desconocido") for fid in lista_ids])

    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_riesgos.append(node)

print(f"✔️ Nodos de riesgos procesados: {len(nodes_riesgos)}")
if nodes_riesgos:
    print("🔍 Ejemplo de nodo de riesgo:")
    print("Texto:", nodes_riesgos[0].text)
    print("Metadatos:", nodes_riesgos[0].metadata)



✔️ Nodos de riesgos procesados: 3
🔍 Ejemplo de nodo de riesgo:
Texto: Riesgo: Caída desde Altura. Este riesgo se refiere a la probabilidad de que una persona caiga desde un nivel superior a uno inferior, resultando en lesiones graves o fatales. Se materializa en situaciones como trabajos en andamios, plataformas elevadas, techos, escaleras verticales, o al transitar cerca de bordes desprotegidos (vanos, aberturas). La altura crítica suele definirse a partir de 1.8 metros, pero cualquier desnivel con potencial de lesión grave es relevante. Las consecuencias pueden variar desde fracturas y traumatismos severos hasta la muerte, además de daños a equipos o interrupción de la operación si la caída afecta infraestructura crítica.

Metadatos: {'clasificacion': None, 'tags': 'caída de altura, impacto, seguridad en altura, trabajo en altura', 'registro_aplicable_a': '', 'aplicabilidad': 'Identificación y evaluación de riesgos críticos', 'area': 'Todas las Áreas Operacionales', 'tiene_causas_aso

In [5]:
# 📌 Celda 6 — Persistencia siguiendo la lógica de test8
# 📌 Celda 6 — R2′ Inserción segura de Riesgos en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json

ids_riesgos = [n.node_id for n in nodes_riesgos]

# 1️⃣ Verificar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_riesgos,
    where=None,
    limit=999999
)
ids_riesgos_chroma = set(chroma_check["ids"])
print(f"Riesgos ya en Chroma: {len(ids_riesgos_chroma)}/{len(ids_riesgos)}")

# 2️⃣ Verificar en NodeStore (en lugar de index.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_riesgos_docstore = {id_ for id_ in ids_riesgos if id_ in ids_docstore}
print(f"Riesgos ya en docstore: {len(ids_riesgos_docstore)}/{len(ids_riesgos)}")

# 3️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_riesgos if n.node_id not in ids_riesgos_chroma]
nodos_faltan_docstore = [n for n in nodes_riesgos if n.node_id not in ids_riesgos_docstore]

print(f"Nodos de riesgos a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de riesgos a insertar en docstore: {len(nodos_faltan_docstore)}")

# 4️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Riesgos insertados en Chroma: {len(nodos_faltan_chroma)}")

# 5️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Riesgos insertados en docstore: {len(nodos_faltan_docstore)}")

# 6️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")

with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")



Riesgos ya en Chroma: 0/3
Riesgos ya en docstore: 0/3
Nodos de riesgos a insertar en Chroma: 3
Nodos de riesgos a insertar en docstore: 3
Riesgos insertados en Chroma: 3
Riesgos insertados en docstore: 3
✅ Persistencia completada. docstore.json actualizado en: C:\\batch001\store_test_yaml2\docstore.json


In [6]:
# 📌 Celda 7 — R3′ Verificación integral (solo Riesgos en este punto)

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrón solo para Riesgos
categorias_regex = {
    "Riesgos": r"^R\d+$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 3

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3



Ejemplo aleatorio de nodos en Chroma (global):

ID: R01
Texto: Riesgo: Caída desde Altura. Este riesgo se refiere a la probabilidad de que una persona caiga desde un nivel superior a uno inferior, resultando en lesiones graves o fatales. Se materializa en situaci ...
Metadatos: {'aplicabilidad': 'Identificación y evaluación de riesgos críticos', 'area': 'Todas las Áreas Operacionales', 'barreras_recuperacion_ids': 'R01BR01, R01BR02, R01BR03, R01BR04', 'barreras_recuperacion_nombres': 'Activación de plan de emergencia y brigada de rescate, Equipo de comunicación de emergencia operativo, Disponibilidad de equipo de primeros auxilios y personal entrenado, Procedimientos claros para aislamiento y estabilización del área', 'causas_asociadas_ids': 'R01CA01, R01CA02, R01CA03, R01CA04, R01CA05, R01CA06', 'causas_asociadas_nombres': 'Bordes o aberturas sin protección, Andamios o plataformas defectuosas o mal montadas, Escaleras portátiles inestables o mal posicionadas, Trabajos sin uso o mal u

In [8]:
# 📌 Celda 8 —  H1′ — Crear nodos de Controles desde YAML (preventivos y de mitigación) con riesgo, causas, factores y consecuencias (ID + nombre)
# 📌 Celda 8 — H1′ — Crear nodos de Controles desde YAML (preventivos y de mitigación) 
# con riesgo, causas, factores y consecuencias (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

yaml_causas = os.path.join(ONTOLOGY_DIR, "03_catalogo_causas_v4.yaml")
with open(yaml_causas, "r", encoding="utf-8") as f:
    causas = yaml.safe_load(f)
mapa_causas = {item["id"]: item.get("nombre", "") for item in causas}

yaml_factores = os.path.join(ONTOLOGY_DIR, "05_catalogo_factores_degradacion_v3.yaml")
with open(yaml_factores, "r", encoding="utf-8") as f:
    factores = yaml.safe_load(f)
mapa_factores = {item["id"]: item.get("nombre", "") for item in factores}

yaml_consecuencias = os.path.join(ONTOLOGY_DIR, "04_catalogo_consecuencias_v6.yaml")
with open(yaml_consecuencias, "r", encoding="utf-8") as f:
    consecuencias = yaml.safe_load(f)
mapa_consecuencias = {item["id"]: item.get("nombre", "") for item in consecuencias}

# === Cargar catálogo de controles ===
yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)

# Campos base para metadatos
campos_permitidos_controles = {
    "aplicabilidad", "tags", "tipo_control", "registro_aplicable_a", 
    "es_critico", "clase_implementacion", "area"
}

# Relaciones comunes a convertir en flags
relaciones_flags_comunes = {
    "riesgo_asociado": "tiene_riesgo_asociado"
}

# Relaciones adicionales según tipo_control
relaciones_flags_preventivos = {
    "causas_que_previene": "tiene_causas_que_previene",
    "factores_degradacion_que_lo_afectan": "tiene_factores_degradacion"
}
relaciones_flags_mitigacion = {
    "consecuencias_que_mitiga": "tiene_consecuencias_que_mitiga"
}

nodes_controles = []
for item in controles:
    # Texto principal con prefijo explícito
    content = f"Control: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # Procesar campos base
    for k in campos_permitidos_controles:
        val = item.get(k, "")
        if isinstance(val, list):
            metadata[k] = ", ".join(val)
        else:
            metadata[k] = val if val is not None else ""

    # Procesar relaciones comunes (riesgo asociado)
    for campo_yaml, campo_flag in relaciones_flags_comunes.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Riesgo asociado (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Según tipo de control ===
    if item.get("tipo_control") == "preventivo":
        # Flags
        for campo_yaml, campo_flag in relaciones_flags_preventivos.items():
            metadata[campo_flag] = bool(item.get(campo_yaml))

        # Causas que previene
        causas_ids = item.get("causas_que_previene", []) or []
        if causas_ids:
            metadata["causas_que_previene_ids"] = ", ".join(causas_ids)
            metadata["causas_que_previene_nombres"] = ", ".join(
                [mapa_causas.get(cid, "Desconocida") for cid in causas_ids]
            )

        # Factores de degradación
        factores_ids = item.get("factores_degradacion_que_lo_afectan", []) or []
        if factores_ids:
            metadata["factores_degradacion_ids"] = ", ".join(factores_ids)
            metadata["factores_degradacion_nombres"] = ", ".join(
                [mapa_factores.get(fid, "Desconocido") for fid in factores_ids]
            )

    elif item.get("tipo_control") == "mitigacion":
        # Flags
        for campo_yaml, campo_flag in relaciones_flags_mitigacion.items():
            metadata[campo_flag] = bool(item.get(campo_yaml))

        # Consecuencias que mitiga
        consecuencias_ids = item.get("consecuencias_que_mitiga", []) or []
        if consecuencias_ids:
            metadata["consecuencias_que_mitiga_ids"] = ", ".join(consecuencias_ids)
            metadata["consecuencias_que_mitiga_nombres"] = ", ".join(
                [mapa_consecuencias.get(cid, "Desconocida") for cid in consecuencias_ids]
            )

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_controles.append(node)

print(f"✔️ Nodos de controles procesados: {len(nodes_controles)}")
if nodes_controles:
    ejemplo = nodes_controles[0]
    print("🔍 Ejemplo de nodo de control:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de controles procesados: 48
🔍 Ejemplo de nodo de control:
ID: R01CC01
Texto: Control: Sistema Personal de Detención de Caídas (SPDC). Implementación y uso obligatorio de arneses de cuerpo completo, líneas de vida con absorbedor de energía y puntos de anclaje certificados. Este sistema debe ser inspeccionado antes de cada uso, garantizar su compatibilidad entre componentes, y ser ajustado adecuadamente por el usuario. Su función principal es detener una caída de forma segura, minimizando las fuerzas de impacto sobre el cuerpo y evitando el contacto con el nivel inferior. Es el último recurso personal para prevenir la caída libre.
Metadatos: {'tags': 'SPDC, arnés, línea de vida, detención de caída, EPP, seguridad en altura, inspección pre-uso, anclaje certificado, fuerza de impacto, prevención de caída libre', 'tipo_control': 'preventivo', 'registro_aplicable_a': 'incidente, observacion', 'es_critico': True, 'aplicabilidad': 'Prevención de incidentes', 'clase_implementacion': 'E

In [9]:
# 📌 Celda 9 —  H2′ — Inserción segura de Controles en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de controles
ids_controles = [n.node_id for n in nodes_controles]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_controles,
    where=None,
    limit=999999
)
ids_controles_chroma = set(chroma_check["ids"])
print(f"Controles ya en Chroma: {len(ids_controles_chroma)}/{len(ids_controles)}")

# 3️⃣ Consultar al docstore SOLO esos IDs (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_controles_docstore = {id_ for id_ in ids_controles if id_ in ids_docstore}
print(f"Controles ya en docstore: {len(ids_controles_docstore)}/{len(ids_controles)}")

# 4️⃣ Filtrar faltantes en Chroma y docstore
nodos_faltan_chroma = [n for n in nodes_controles if n.node_id not in ids_controles_chroma]
nodos_faltan_docstore = [n for n in nodes_controles if n.node_id not in ids_controles_docstore]

print(f"Nodos de controles a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de controles a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Controles insertados en Chroma: {len(nodos_faltan_chroma)}")

# 6️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Controles insertados en docstore: {len(nodos_faltan_docstore)}")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar manualmente el docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")



Controles ya en Chroma: 0/48
Controles ya en docstore: 0/48
Nodos de controles a insertar en Chroma: 48
Nodos de controles a insertar en docstore: 48
Controles insertados en Chroma: 48
Controles insertados en docstore: 48
✅ Persistencia completada. docstore.json actualizado en: C:\\batch001\store_test_yaml2\docstore.json


In [10]:
# 📌 Celda 10 - H3′ — Verificación integral con Controles incluidos
# 📌 Celda H3′ — Verificación integral con Controles incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (Riesgos + Controles)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CC|CP|CM)\d+$"  # controles críticos, preventivos, mitigación
}

# 3️⃣ Conteo por categoría (usando storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 5️⃣ Muestreo específico de Controles
ids_controles_chroma = [id_ for id_ in ids_chroma if re.match(r"^R\d+(CC|CP|CM)\d+$", id_)]
if ids_controles_chroma:
    sample_ctrl = random.sample(ids_controles_chroma, min(3, len(ids_controles_chroma)))
    print("\nEjemplo específico de nodos de Controles en Chroma:")

    for sid in sample_ctrl:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 51

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48



Ejemplo aleatorio de nodos en Chroma (global):

ID: R03CC02
Texto: Control: Mantenimiento preventivo y correctivo de instalaciones y equipos eléctricos. Programa sistemático de inspección, pruebas y reparaciones de cables, conexiones, tableros, transformadores y equi ...
Metadatos: {'aplicabilidad': 'Prevención de incidentes', 'area': 'Todas las Áreas Operacionales', 'causas_que_previene_ids': 'R03CA02, R03CA06', 'causas_que_previene_nombres': 'Cables eléctricos dañados, conexiones expuestas o improvisadas, Instalaciones eléctricas defectuosas, expuestas o sin mantenimiento', 'clase_implementacion': 'técnico', 'doc_id': 'None', 'document_id': 'None', 'es_critico': True, 'factores_degradacion_ids': 'R03FD01, R03FD02, R03FD03', 'factores_degradacion_nombres': 'Aislamiento eléctrico deteriorado/expuesto, Bloqueo y etiquetado (LOTO) deficiente, Herramientas no verificadas tras traslado entre turnos', 'ref_doc_id': 'None', 'registro_aplicable_a': 'incidente, observacion', 'riesgo_asociado_

In [4]:
# 📌 Celda 11 - I1′ — Crear nodos de Causas desde YAML con riesgo y controles que previenen (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item["nombre"] for item in riesgos}

yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)
mapa_controles = {item["id"]: item["nombre"] for item in controles}

# === Cargar catálogo de causas ===
yaml_causas = os.path.join(ONTOLOGY_DIR, "03_catalogo_causas_v4.yaml")
with open(yaml_causas, "r", encoding="utf-8") as f:
    causas = yaml.safe_load(f)

# Campos base para metadatos
campos_permitidos_causas = {
    "aplicabilidad", "tags", "tipo_causa", "registro_aplicable_a", "area"
}

# Relaciones a convertir en flags
relaciones_flags_causas = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "controles_que_previenen": "tiene_controles_que_previenen"
}

nodes_causas = []
for item in causas:
    # Texto principal explícito
    content = f"Causa: {item.get('nombre', '')}. {item.get('descripcion', '')}"
    metadata = {}

    # Procesar campos base
    for k in campos_permitidos_causas:
        if k == "tags":
            tags = item.get("tags", [])
            metadata[k] = ", ".join(tags) if isinstance(tags, list) else str(tags)
        elif k == "registro_aplicable_a":
            lista = item.get(k, [])
            metadata[k] = ", ".join(lista) if isinstance(lista, list) else str(lista)
        else:
            metadata[k] = item.get(k)

    # Procesar relaciones como flags
    for campo_yaml, campo_flag in relaciones_flags_causas.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Asociación explícita con Riesgo (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Asociación explícita con Controles que previenen (IDs + nombres) ===
    controles_ids = item.get("controles_que_previenen", [])
    if controles_ids:
        metadata["controles_que_previenen_ids"] = ", ".join(controles_ids)
        metadata["controles_que_previenen_nombres"] = ", ".join(
            [mapa_controles.get(cid, "Desconocido") for cid in controles_ids]
        )

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_causas.append(node)

print(f"✔️ Nodos de causas procesados: {len(nodes_causas)}")
if nodes_causas:
    print("🔍 Ejemplo de nodo de causa:")
    print("Texto:", nodes_causas[0].text)
    print("Metadatos:", nodes_causas[0].metadata)


✔️ Nodos de causas procesados: 18
🔍 Ejemplo de nodo de causa:
Texto: Causa: Bordes o aberturas sin protección. Ausencia total o parcial de sistemas de protección perimetral (barandas, rodapiés, cubiertas) en bordes de plataformas, vanos, aberturas en el piso, o desniveles significativos. Esto permite que una persona, herramienta o material caiga sin contención. Puede variar en la altura del desnivel, el tipo de borde (recto, curvo), la presencia o ausencia de señalización, y el tipo de actividad que se realiza cerca del borde. Ejemplos incluyen la falta de barandas en una plataforma de trabajo, una abertura sin cubrir en un piso, o un desnivel sin rodapiés en una pasarela.

Metadatos: {'aplicabilidad': 'Prevención de incidentes', 'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente, observacion', 'tipo_causa': 'Fallas de infraestructura', 'tags': 'seguridad perimetral, barandas, rodapiés, infraestructura, diseño, protección de borde, apertura, vano, desnivel, con

In [5]:
# 📌 Celda 12 - I2′ — Inserción segura de Causas en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de causas
ids_causas = [n.node_id for n in nodes_causas]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_causas,
    where=None,
    limit=999999
)
ids_causas_chroma = set(chroma_check["ids"])
print(f"Causas ya en Chroma: {len(ids_causas_chroma)}/{len(ids_causas)}")

# 3️⃣ Consultar al docstore SOLO esos IDs (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_causas_docstore = {id_ for id_ in ids_causas if id_ in ids_docstore}
print(f"Causas ya en docstore: {len(ids_causas_docstore)}/{len(ids_causas)}")

# 4️⃣ Filtrar faltantes en Chroma y docstore
nodos_faltan_chroma = [n for n in nodes_causas if n.node_id not in ids_causas_chroma]
nodos_faltan_docstore = [n for n in nodes_causas if n.node_id not in ids_causas_docstore]

print(f"Nodos de causas a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de causas a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Causas insertadas en Chroma: {len(nodos_faltan_chroma)}")

# 6️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Causas insertadas en docstore: {len(nodos_faltan_docstore)}")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar manualmente el docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Causas ya en Chroma: 0/18
Causas ya en docstore: 0/18
Nodos de causas a insertar en Chroma: 18
Nodos de causas a insertar en docstore: 18
Causas insertadas en Chroma: 18
Causas insertadas en docstore: 18
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [6]:
# 📌 Celda 13 — Verificación de nodos de causas en docstore.json
# 📌 Celda I3′ — Verificación integral con Causas incluidas

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (Riesgos + Controles + Causas)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CC|CP|CM)\d+$",
    "Causas": r"^R\d+CA\d+$"
}

# 3️⃣ Conteo por categoría (Chroma + Docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 5️⃣ Muestreo específico de Causas
ids_causas_chroma = [id_ for id_ in ids_chroma if re.match(r"^R\d+CA\d+$", id_)]
print(f"\nTotal de Causas en Chroma: {len(ids_causas_chroma)}")

if ids_causas_chroma:
    sample_causas = random.sample(ids_causas_chroma, min(3, len(ids_causas_chroma)))
    print("\nEjemplo específico de nodos de Causas en Chroma:")

    for sid in sample_causas:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})



Total de IDs en Chroma: 69

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18



Ejemplo aleatorio de nodos en Chroma (global):

ID: R03CA05
Texto: Causa: Falta de capacitación o competencia en seguridad eléctrica. Personal que realiza trabajos con riesgo eléctrico sin la formación teórica y práctica necesaria sobre los peligros de la electricida ...
Metadatos: {'aplicabilidad': 'Prevención de incidentes', 'area': 'Todas las Áreas Operacionales', 'controles_que_previenen_ids': 'R03CP05, R03CC05', 'controles_que_previenen_nombres': 'Cursos de concientización básica en seguridad eléctrica, Capacitación y certificación de personal para trabajos eléctricos', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'registro_aplicable_a': 'incidente, observacion', 'riesgo_asociado_id': 'R03', 'riesgo_asociado_nombre': 'Contacto con Energía Eléctrica', 'tags': 'personal calificado, acreditación, capacitación, entrenamiento, eléctrico, competencia, factor humano', 'tiene_controles_que_previenen': True, 'tiene_riesgo_asociado': True, 'tipo_causa': 'Factores humanos/

In [7]:
# 📌 Celda 14 — Crear nodos de consecuencias desde YAML
# 📌 Celda 14 - J1′ — Crear nodos de Consecuencias desde YAML con riesgo, controles y barreras asociadas (ID + nombre)
# 📌 Celda 14 - J1′ — Crear nodos de Consecuencias desde YAML con riesgo, controles y barreras asociadas (ID + nombre + tipo_control de controles)

import yaml
from llama_index.core.schema import TextNode
import os

# === Cargar catálogos de referencia ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)
# Guardamos nombre + tipo_control
mapa_controles = {
    item["id"]: {
        "nombre": item.get("nombre", ""),
        "tipo_control": item.get("tipo_control", "")
    }
    for item in controles
}

yaml_barreras = os.path.join(ONTOLOGY_DIR, "06_catalogo_barreras_recuperacion_v3.yaml")
with open(yaml_barreras, "r", encoding="utf-8") as f:
    barreras = yaml.safe_load(f)
mapa_barreras = {item["id"]: item.get("nombre", "") for item in barreras}

# === Cargar catálogo de consecuencias ===
yaml_consecuencias = os.path.join(ONTOLOGY_DIR, "04_catalogo_consecuencias_v6.yaml")
with open(yaml_consecuencias, "r", encoding="utf-8") as f:
    consecuencias = yaml.safe_load(f)

# Campos base para metadatos
campos_permitidos_consecuencias = {
    "aplicabilidad", "tags", "tipo_consecuencia", "severidad_consecuencia",
    "registro_aplicable_a", "area"
}

# Relaciones a convertir en flags
relaciones_flags_consecuencias = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "controles_mitigacion_asociados": "tiene_controles_mitigacion_asociados",
    "barreras_recuperacion_asociadas": "tiene_barreras_recuperacion_asociadas"
}

nodes_consecuencias = []
for item in consecuencias:
    # Texto principal con prefijo explícito
    nombre = item.get("nombre", "")
    descripcion = item.get("descripcion", "")
    content = f"Consecuencia: {nombre}. {descripcion}".strip()

    metadata = {}

    # Procesar campos base
    for k in campos_permitidos_consecuencias:
        val = item.get(k, "")
        if isinstance(val, list):
            metadata[k] = ", ".join(val)
        else:
            metadata[k] = val if val is not None else ""

    # Procesar relaciones como flags
    for campo_yaml, campo_flag in relaciones_flags_consecuencias.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Riesgo asociado (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Controles de mitigación (IDs + nombres + tipos) ===
    controles_ids = item.get("controles_mitigacion_asociados", []) or []
    if controles_ids:
        metadata["controles_mitigacion_asociados_ids"] = ", ".join(controles_ids)
        metadata["controles_mitigacion_asociados_nombres"] = ", ".join(
            [mapa_controles.get(cid, {}).get("nombre", "Desconocido") for cid in controles_ids]
        )
        metadata["controles_mitigacion_asociados_tipos"] = ", ".join(
            [mapa_controles.get(cid, {}).get("tipo_control", "Desconocido") for cid in controles_ids]
        )

    # === Barreras de recuperación (IDs + nombres) ===
    barreras_ids = item.get("barreras_recuperacion_asociadas", []) or []
    if barreras_ids:
        metadata["barreras_recuperacion_asociadas_ids"] = ", ".join(barreras_ids)
        metadata["barreras_recuperacion_asociadas_nombres"] = ", ".join(
            [mapa_barreras.get(bid, "Desconocida") for bid in barreras_ids]
        )

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_consecuencias.append(node)

print(f"✔️ Nodos de consecuencias procesados: {len(nodes_consecuencias)}")
if nodes_consecuencias:
    ejemplo = nodes_consecuencias[0]
    print("🔍 Ejemplo de nodo de consecuencia:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de consecuencias procesados: 18
🔍 Ejemplo de nodo de consecuencia:
ID: R01CON01
Texto: Consecuencia: Lesiones graves. Fracturas óseas múltiples, traumatismos craneoencefálicos y contusiones profundas que requieren atención médica de urgencia. Estas lesiones suelen derivar en hospitalización prolongada, intervenciones quirúrgicas y tiempo de inactividad laboral significativo. El impacto puede incluir secuelas a largo plazo como limitaciones de movilidad o déficit neurológico.
Metadatos: {'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente', 'tipo_consecuencia': 'Salud_Persona', 'severidad_consecuencia': 'ALTA', 'aplicabilidad': 'Evaluación de consecuencias', 'tags': 'gravedad, trauma, incapacidad, lesiones', 'tiene_riesgo_asociado': True, 'tiene_controles_mitigacion_asociados': True, 'tiene_barreras_recuperacion_asociadas': True, 'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'controles_mitigacion_asociados_ids': 'R01CM01, R

In [8]:
# 📌 Celda 15 — Insertar consecuencias y persistir en Chroma y docstore.json (flujo seguro)
# 📌 Celda 15 - J2′ — Inserción segura de Consecuencias en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de consecuencias
ids_consecuencias = [n.node_id for n in nodes_consecuencias]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_consecuencias,
    where=None,
    limit=999999
)
ids_consecuencias_chroma = set(chroma_check["ids"])
print(f"Consecuencias ya en Chroma: {len(ids_consecuencias_chroma)}/{len(ids_consecuencias)}")

# 3️⃣ Consultar al docstore SOLO esos IDs (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_consecuencias_docstore = {id_ for id_ in ids_consecuencias if id_ in ids_docstore}
print(f"Consecuencias ya en docstore: {len(ids_consecuencias_docstore)}/{len(ids_consecuencias)}")

# 4️⃣ Filtrar faltantes en Chroma y docstore
nodos_faltan_chroma = [n for n in nodes_consecuencias if n.node_id not in ids_consecuencias_chroma]
nodos_faltan_docstore = [n for n in nodes_consecuencias if n.node_id not in ids_consecuencias_docstore]

print(f"Nodos de consecuencias a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de consecuencias a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Consecuencias insertadas en Chroma: {len(nodos_faltan_chroma)}")

# 6️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Consecuencias insertadas en docstore: {len(nodos_faltan_docstore)}")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar manualmente el docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")



Consecuencias ya en Chroma: 0/18
Consecuencias ya en docstore: 0/18
Nodos de consecuencias a insertar en Chroma: 18
Nodos de consecuencias a insertar en docstore: 18
Consecuencias insertadas en Chroma: 18
Consecuencias insertadas en docstore: 18
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [9]:
# 📌 Celda 16 — J3′ — Verificación integral con Consecuencias incluidas

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (Riesgos + Controles + Causas + Consecuencias)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CC|CP|CM)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$"
}

# 3️⃣ Conteo por categoría (Chroma + Docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 5️⃣ Muestreo específico de Consecuencias
ids_consecuencias_chroma = [id_ for id_ in ids_chroma if re.match(r"^R\d+CON\d+$", id_)]
print(f"\nTotal de Consecuencias en Chroma: {len(ids_consecuencias_chroma)}")

if ids_consecuencias_chroma:
    sample_consecuencias = random.sample(ids_consecuencias_chroma, min(3, len(ids_consecuencias_chroma)))
    print("\nEjemplo específico de nodos de Consecuencias en Chroma:")

    for sid in sample_consecuencias:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 87

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18



Ejemplo aleatorio de nodos en Chroma (global):

ID: R01CC01
Texto: Control: Sistema Personal de Detención de Caídas (SPDC). Implementación y uso obligatorio de arneses de cuerpo completo, líneas de vida con absorbedor de energía y puntos de anclaje certificados. Este ...
Metadatos: {'aplicabilidad': 'Prevención de incidentes', 'area': 'Todas las Áreas Operacionales', 'causas_que_previene_ids': 'R01CA04', 'causas_que_previene_nombres': 'Trabajos sin uso o mal uso de SPDC/EPP de altura', 'clase_implementacion': 'EPP', 'doc_id': 'None', 'document_id': 'None', 'es_critico': True, 'factores_degradacion_ids': 'R01FD01, R01FD04', 'factores_degradacion_nombres': 'Etiquetado de inspección vencido o inexistente, Superficie de trabajo resbaladiza/irregular', 'ref_doc_id': 'None', 'registro_aplicable_a': 'incidente, observacion', 'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'tags': 'SPDC, arnés, línea de vida, detención de caída, EPP, seguridad en altura, inspecció

In [ ]:
# revisar desde aqui para adelante

In [10]:
# 📌 Celda 17 — Crear nodos de factores de degradación desde YAML con riesgo explícito (ID + nombre)

# 📌 Celda 17 — Crear nodos de Factores de Degradación desde YAML con riesgo y controles asociados (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de riesgos para mapear IDs -> nombres ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# === Cargar catálogo de controles para mapear IDs -> nombres ===
yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)
mapa_controles = {item["id"]: item.get("nombre", "") for item in controles}

# === Cargar catálogo de factores de degradación ===
yaml_factores = os.path.join(ONTOLOGY_DIR, "05_catalogo_factores_degradacion_v3.yaml")
with open(yaml_factores, "r", encoding="utf-8") as f:
    factores = yaml.safe_load(f)

# Campos base para metadatos
campos_permitidos_factores = {
    "aplicabilidad", "tags", "area", "registro_aplicable_a", "tipo_factor"
}

# Relaciones a convertir en flags booleanos
relaciones_flags_factores = {
    "riesgo_asociado": "tiene_riesgo_asociado",
    "controles_asociados_criticos_y_preventivos": "tiene_controles_asociados"
}

nodes_factores = []
for item in factores:
    # Texto principal con prefijo explícito
    content = f"Factor de Degradación: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # Procesar campos permitidos
    for k in campos_permitidos_factores:
        val = item.get(k, "")
        if isinstance(val, list):
            metadata[k] = ", ".join(val)
        else:
            metadata[k] = val if val is not None else ""

    # Procesar relaciones como flags booleanos
    for campo_yaml, campo_flag in relaciones_flags_factores.items():
        metadata[campo_flag] = bool(item.get(campo_yaml))

    # === Asociación explícita con riesgo (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Asociación explícita con controles (IDs + nombres) ===
    controles_ids = item.get("controles_asociados_criticos_y_preventivos", []) or []
    if controles_ids:
        metadata["controles_asociados_ids"] = ", ".join(controles_ids)
        metadata["controles_asociados_nombres"] = ", ".join(
            [mapa_controles.get(cid, "Desconocido") for cid in controles_ids]
        )

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_factores.append(node)

print(f"✔️ Nodos de factores de degradación procesados: {len(nodes_factores)}")
if nodes_factores:
    ejemplo = nodes_factores[0]
    print("🔍 Ejemplo de nodo de factor de degradación:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de factores de degradación procesados: 12
🔍 Ejemplo de nodo de factor de degradación:
ID: R01FD01
Texto: Factor de Degradación: Etiquetado de inspección vencido o inexistente. Ausencia o expiración de etiquetas e indicadores de inspección en equipos y estructuras de protección. Provoca incertidumbre sobre la integridad del sistema de control y dificulta la identificación de elementos con mantenimiento vencido, aumentando la probabilidad de fallas no detectadas y accidentes de caída.
Metadatos: {'tipo_factor': 'Administrativo/Mantenimiento', 'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente, observacion', 'aplicabilidad': 'Prevención de condiciones degradadas', 'tags': 'inspección, etiquetado, vencido, mantenimiento, procedimiento', 'tiene_riesgo_asociado': True, 'tiene_controles_asociados': True, 'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'controles_asociados_ids': 'R01CC03, R01CP06', 'controles_asociados_nombres': 'P

In [11]:
# 📌 Celda 18 — Inserción segura de factores de degradación en docstore + Chroma con verificación previa

# 📌 Celda 18 — Inserción segura de Factores de Degradación en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de factores de degradación
ids_factores = [n.node_id for n in nodes_factores]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_factores,
    where=None,
    limit=999999
)
ids_factores_chroma = set(chroma_check["ids"])
print(f"Factores ya en Chroma: {len(ids_factores_chroma)}/{len(ids_factores)}")

# 3️⃣ Consultar al docstore SOLO esos IDs (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_factores_docstore = {id_ for id_ in ids_factores if id_ in ids_docstore}
print(f"Factores ya en docstore: {len(ids_factores_docstore)}/{len(ids_factores)}")

# 4️⃣ Filtrar faltantes en Chroma y docstore
nodos_faltan_chroma = [n for n in nodes_factores if n.node_id not in ids_factores_chroma]
nodos_faltan_docstore = [n for n in nodes_factores if n.node_id not in ids_factores_docstore]

print(f"Nodos de factores a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de factores a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma si faltan
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Factores de degradación insertados en Chroma: {len(nodos_faltan_chroma)}")

# 6️⃣ Insertar en docstore si faltan
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Factores de degradación insertados en docstore: {len(nodos_faltan_docstore)}")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar manualmente el docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Factores ya en Chroma: 0/12
Factores ya en docstore: 0/12
Nodos de factores a insertar en Chroma: 12
Nodos de factores a insertar en docstore: 12
Factores de degradación insertados en Chroma: 12
Factores de degradación insertados en docstore: 12
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [12]:
# 📌 Celda 19 — Verificación integral de nodos en docstore y Chroma

# 📌 Celda 19 — Verificación integral con Factores de Degradación incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo factores)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$"
}

# 3️⃣ Conteo por categoría en Chroma y docstore
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma:")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Factores de Degradación
ids_factores_chroma = [id_ for id_ in ids_chroma if re.match(r"^R\d+FD\d+$", id_)]
print(f"\nTotal de Factores de Degradación en Chroma: {len(ids_factores_chroma)}")

if ids_factores_chroma:
    sample_factores = random.sample(ids_factores_chroma, min(3, len(ids_factores_chroma)))
    print("\nEjemplo específico de nodos de Factores de Degradación en Chroma:")

    for sid in sample_factores:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})



Total de IDs en Chroma: 99

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12



Ejemplo aleatorio de nodos en Chroma:

ID: R01CON01
Texto: Consecuencia: Lesiones graves. Fracturas óseas múltiples, traumatismos craneoencefálicos y contusiones profundas que requieren atención médica de urgencia. Estas lesiones suelen derivar en hospitaliza ...
Metadatos: {'aplicabilidad': 'Evaluación de consecuencias', 'area': 'Todas las Áreas Operacionales', 'barreras_recuperacion_asociadas_ids': 'R01BR01, R01BR03, R03BR02, R03BR03', 'barreras_recuperacion_asociadas_nombres': 'Activación de plan de emergencia y brigada de rescate, Disponibilidad de equipo de primeros auxilios y personal entrenado, Reanimación cardiopulmonar inmediata y desfibrilación, Activación de protocolo de emergencia eléctrica', 'controles_mitigacion_asociados_ids': 'R01CM01, R01CM03, R03CM03', 'controles_mitigacion_asociados_nombres': 'Aplicación de primeros auxilios y evacuación de emergencia, Notificación a autoridades y equipo de emergencia (interno/externo), Suministro de atención médica de emergencia y 

In [13]:
# 📌 Celda 20 — Crear nodos de barreras de recuperación desde YAML con riesgo explícito (ID + nombre)
# 📌 Celda 20 — Crear nodos de Barreras de Recuperación desde YAML con riesgo y consecuencias asociadas (ID + nombre)

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de riesgos para mapear IDs -> nombres ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# === Cargar catálogo de consecuencias para mapear IDs -> nombres ===
yaml_consecuencias = os.path.join(ONTOLOGY_DIR, "04_catalogo_consecuencias_v6.yaml")
with open(yaml_consecuencias, "r", encoding="utf-8") as f:
    consecuencias = yaml.safe_load(f)
mapa_consecuencias = {item["id"]: item.get("nombre", "") for item in consecuencias}

# === Cargar catálogo de barreras de recuperación ===
yaml_barreras = os.path.join(ONTOLOGY_DIR, "06_catalogo_barreras_recuperacion_v3.yaml")
with open(yaml_barreras, "r", encoding="utf-8") as f:
    barreras = yaml.safe_load(f)

# Campos base para metadatos
campos_permitidos_barreras = {
    "aplicabilidad", "tags", "area", "registro_aplicable_a", "tipo_barrera"
}

nodes_barreras = []
for item in barreras:
    # Texto principal con prefijo explícito
    content = f"Barrera de Recuperación: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # Procesar campos base
    for k in campos_permitidos_barreras:
        val = item.get(k, "")
        if isinstance(val, list):
            metadata[k] = ", ".join(val)
        else:
            metadata[k] = val if val is not None else ""

    # === Flags ===
    metadata["tiene_riesgo_asociado"] = bool(item.get("riesgo_asociado"))
    metadata["tiene_consecuencias_asociadas"] = bool(item.get("consecuencias_asociadas"))

    # === Asociación explícita con riesgo (ID + nombre) ===
    riesgo_id = item.get("riesgo_asociado")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Asociación explícita con consecuencias (IDs + nombres) ===
    consecuencias_ids = item.get("consecuencias_asociadas", []) or []
    if consecuencias_ids:
        metadata["consecuencias_asociadas_ids"] = ", ".join(consecuencias_ids)
        metadata["consecuencias_asociadas_nombres"] = ", ".join(
            [mapa_consecuencias.get(cid, "Desconocida") for cid in consecuencias_ids]
        )

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_barreras.append(node)

print(f"✔️ Nodos de barreras de recuperación procesados: {len(nodes_barreras)}")
if nodes_barreras:
    ejemplo = nodes_barreras[0]
    print("🔍 Ejemplo de nodo de barrera de recuperación:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de barreras de recuperación procesados: 11
🔍 Ejemplo de nodo de barrera de recuperación:
ID: R01BR01
Texto: Barrera de Recuperación: Activación de plan de emergencia y brigada de rescate. Plan de emergencia formalizado que moviliza a la brigada de rescate entrenada para atender incidentes de caída de altura. Incluye protocolos de notificación inmediata, rutas de acceso seguras y equipos de rescate especializados. Permite reducir el tiempo de respuesta a menos de 10 minutos y aumentar la probabilidad de supervivencia y atención oportuna.
Metadatos: {'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente', 'tags': 'plan de emergencia, brigada de rescate, respuesta inmediata, protocolos', 'aplicabilidad': 'Recuperación tras incidente', 'tipo_barrera': 'Administrativo/Respuesta a Emergencias', 'tiene_riesgo_asociado': True, 'tiene_consecuencias_asociadas': True, 'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'consecuencias_asociad

In [14]:
# 📌 Celda 21 — Inserción segura de barreras de recuperación en docstore + Chroma con verificación previa
# 📌 Celda 21 — Inserción segura de Barreras de Recuperación en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer todos los IDs de barreras
ids_barreras = [n.node_id for n in nodes_barreras]

# 2️⃣ Consultar a Chroma SOLO esos IDs
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_barreras,
    where=None,
    limit=999999
)
ids_barreras_chroma = set(chroma_check["ids"])
print(f"Barreras ya en Chroma: {len(ids_barreras_chroma)}/{len(ids_barreras)}")

# 3️⃣ Consultar al docstore SOLO esos IDs (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_barreras_docstore = {id_ for id_ in ids_barreras if id_ in ids_docstore}
print(f"Barreras ya en docstore: {len(ids_barreras_docstore)}/{len(ids_barreras)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_barreras if n.node_id not in ids_barreras_chroma]
nodos_faltan_docstore = [n for n in nodes_barreras if n.node_id not in ids_barreras_docstore]

print(f"Nodos de barreras a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de barreras a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print(f"Barreras de recuperación insertadas en Chroma: {len(nodos_faltan_chroma)}")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print(f"Barreras de recuperación insertadas en docstore: {len(nodos_faltan_docstore)}")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar manualmente el docstore.json
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")



Barreras ya en Chroma: 0/11
Barreras ya en docstore: 0/11
Nodos de barreras a insertar en Chroma: 11
Nodos de barreras a insertar en docstore: 11
Barreras de recuperación insertadas en Chroma: 11
Barreras de recuperación insertadas en docstore: 11
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [15]:
# 📌 Celda 22 — Verificación integral con barreras incluidas
# 📌 Celda 22 — Verificación integral con Barreras de Recuperación incluidas

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma:")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 5️⃣ Muestreo específico de Barreras
ids_barreras_chroma = [id_ for id_ in ids_chroma if re.match(r"^R\d+BR\d+$", id_)]
print(f"\nTotal de Barreras de Recuperación en Chroma: {len(ids_barreras_chroma)}")

if ids_barreras_chroma:
    sample_barreras = random.sample(ids_barreras_chroma, min(3, len(ids_barreras_chroma)))
    print("\nEjemplo específico de nodos de Barreras de Recuperación en Chroma:")

    for sid in sample_barreras:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})



Total de IDs en Chroma: 110

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11



Ejemplo aleatorio de nodos en Chroma:

ID: R03
Texto: Riesgo: Contacto con Energía Eléctrica. Este riesgo se refiere al contacto directo o indirecto de una persona con energía eléctrica, ya sea por intervención en equipos energizados, fallas de aislamien ...
Metadatos: {'aplicabilidad': 'Identificación y evaluación de riesgos críticos', 'area': 'Todas las Áreas Operacionales', 'barreras_recuperacion_ids': 'R03BR01, R03BR02, R03BR03, R03BR04', 'barreras_recuperacion_nombres': 'Desconexión rápida de la fuente de energía, Reanimación cardiopulmonar inmediata y desfibrilación, Activación de protocolo de emergencia eléctrica, Evaluación y reparación de equipos afectados post-evento', 'causas_asociadas_ids': 'R03CA01, R03CA02, R03CA03, R03CA04, R03CA05, R03CA06', 'causas_asociadas_nombres': 'No desenergizar o verificar ausencia de tensión (LOTO incompleto), Cables eléctricos dañados, conexiones expuestas o improvisadas, Falta de EPP dieléctrico o uso de EPP inadecuado/dañado, Acceso no auto

In [16]:
# 📌 Celda 23 - AR1 — Crear nodos de áreas operacionales desde YAML
# 📌 Celda 23 - AR1 — Crear nodos de Áreas Operacionales desde YAML

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de áreas operacionales ===
yaml_areas = os.path.join(ONTOLOGY_DIR, "13_catalogo_areas_operacionales_v1.yaml")
with open(yaml_areas, "r", encoding="utf-8") as f:
    areas = yaml.safe_load(f)

nodes_areas = []
for item in areas:
    # Texto principal con prefijo explícito
    content = f"Área Operacional: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # Procesar campos base
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")

    # Convertir listas a string
    tags = item.get("tags", [])
    metadata["tags"] = ", ".join(tags) if isinstance(tags, list) else str(tags)

    registro = item.get("registro_aplicable_a", [])
    metadata["registro_aplicable_a"] = ", ".join(registro) if isinstance(registro, list) else str(registro)

    # Flags
    metadata["tiene_tags"] = bool(item.get("tags"))
    metadata["tiene_registro_aplicable_a"] = bool(item.get("registro_aplicable_a"))

    # Guardar también el id para trazabilidad (además de node_id)
    metadata["id_area"] = item.get("id", "")

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_areas.append(node)

print(f"✔️ Nodos de áreas operacionales procesados: {len(nodes_areas)}")
if nodes_areas:
    ejemplo = nodes_areas[0]
    print("🔍 Ejemplo de nodo de área operacional:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de áreas operacionales procesados: 5
🔍 Ejemplo de nodo de área operacional:
ID: MRA
Texto: Área Operacional: Mina Rajo Abierto. Área principal de extracción de mineral y material estéril en superficie. Involucra la perforación, voladura, carguío y transporte de grandes volúmenes de roca utilizando maquinaria pesada (camiones de alto tonelaje, palas hidráulicas y perforadoras). Es fundamental para la alimentación de mineral a planta y la conformación del diseño final de la mina.
Metadatos: {'area': 'Mina Rajo Abierto', 'aplicabilidad': 'Definición de áreas operacionales', 'tags': 'mina, rajo_abierto, extracción_superficial, perforación, voladura, carguío, transporte_pesado, camiones_mina, palas_excavadoras, bancos_mina, operaciones_mineras, minería_superficial, equipo_pesado, movimiento_tierra', 'registro_aplicable_a': 'incidente, observacion', 'tiene_tags': True, 'tiene_registro_aplicable_a': True, 'id_area': 'MRA'}


In [17]:
# 📌 Celda 24 - AR2 — Inserción segura de áreas en docstore + Chroma
# 📌 Celda 24 - AR2 — Inserción segura de Áreas en docstore + Chroma con verificación previa

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_areas = [n.node_id for n in nodes_areas]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_areas,
    where=None,
    limit=999999
)
ids_areas_chroma = set(chroma_check["ids"])
print(f"Áreas ya en Chroma: {len(ids_areas_chroma)}/{len(ids_areas)}")

# 3️⃣ Consultar en docstore (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_areas_docstore = {id_ for id_ in ids_areas if id_ in ids_docstore}
print(f"Áreas ya en docstore: {len(ids_areas_docstore)}/{len(ids_areas)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_areas if n.node_id not in ids_areas_chroma]
nodos_faltan_docstore = [n for n in nodes_areas if n.node_id not in ids_areas_docstore]

print(f"Nodos de áreas a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de áreas a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Áreas insertadas en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Áreas insertadas en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")



Áreas ya en Chroma: 0/5
Áreas ya en docstore: 0/5
Nodos de áreas a insertar en Chroma: 5
Nodos de áreas a insertar en docstore: 5
Áreas insertadas en Chroma.
Áreas insertadas en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [20]:
# 📌 Celda 25 - AR3 — Verificación integral con áreas incluidas
# 📌 Celda 25 - AR3 — Verificación integral con Áreas incluidas (ajustada a acrónimos)

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (ajuste para áreas con acrónimos)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Áreas Operacionales
ids_areas_chroma = [id_ for id_ in ids_chroma if re.match(r"^(MRA|CHP|TME|STM|PLC)$", id_)]
print(f"\nTotal de Áreas Operacionales en Chroma: {len(ids_areas_chroma)}")

if ids_areas_chroma:
    sample_areas = random.sample(ids_areas_chroma, min(3, len(ids_areas_chroma)))
    print("\nEjemplo específico de nodos de Áreas Operacionales en Chroma:")

    for sid in sample_areas:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})



Total de IDs en Chroma: 115

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Áreas Operacionales,5,5



Ejemplo aleatorio de nodos en Chroma (global):

ID: CHP
Texto: Área Operacional: Planta de Chancado Primario. Instalación industrial donde el mineral grueso proveniente de la mina es sometido a la primera etapa de reducción de tamaño. Utiliza chancadoras de gran  ...
Metadatos: {'aplicabilidad': 'Definición de áreas operacionales', 'area': 'Planta de Chancado Primario', 'doc_id': 'None', 'document_id': 'None', 'id_area': 'CHP', 'ref_doc_id': 'None', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'chancado_primario, trituración, reducción_tamaño, mineral_grueso, chancadoras_primarias, correas_transportadoras, tolvas_alimentación, planta_procesamiento, conminución, ruido, polvo', 'tiene_registro_aplicable_a': True, 'tiene_tags': True}

ID: R01CP06
Texto: Control: Delimitación de zonas de riesgo y control de acceso. Uso de barreras físicas temporales (vallas, cintas, conos) y personal de vigilancia para restringir el acceso a áreas con riesgo de caída, ...
Metadatos: {'aplicab

In [22]:
# 📌 Celda 26 - TA1 — Crear nodos de Tareas con nombre de riesgos y roles
# 📌 Celda 26 - TA1 — Crear nodos de Tareas con nombre de riesgos, roles, controles, factores y EPP

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
# Roles
yaml_roles = os.path.join(ONTOLOGY_DIR, "10_catalogo_roles_v3.yaml")
with open(yaml_roles, "r", encoding="utf-8") as f:
    roles = yaml.safe_load(f)
mapa_roles = {item["id"]: item.get("nombre", "") for item in roles}

# Riesgos
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# Controles
yaml_controles = os.path.join(ONTOLOGY_DIR, "02_catalogo_controles_v6.yaml")
with open(yaml_controles, "r", encoding="utf-8") as f:
    controles = yaml.safe_load(f)
mapa_controles = {item["id"]: item.get("nombre", "") for item in controles}

# Factores de Exposición
yaml_factores = os.path.join(ONTOLOGY_DIR, "11_catalogo_factores_exposicion_v3.yaml")
with open(yaml_factores, "r", encoding="utf-8") as f:
    factores = yaml.safe_load(f)
mapa_factores = {item["id"]: item.get("nombre", "") for item in factores}

# EPP
yaml_epp = os.path.join(ONTOLOGY_DIR, "14_catalogo_epp_v3.yaml")
with open(yaml_epp, "r", encoding="utf-8") as f:
    epp_list = yaml.safe_load(f)
mapa_epp = {item["id"]: item.get("nombre", "") for item in epp_list}

# === Cargar catálogo de tareas ===
yaml_tareas = os.path.join(ONTOLOGY_DIR, "12_catalogo_tareas_v1.yaml")
with open(yaml_tareas, "r", encoding="utf-8") as f:
    tareas = yaml.safe_load(f)

nodes_tareas = []
for item in tareas:
    # Texto principal con prefijo explícito
    content = f"Tarea: {item.get('nombre_tarea', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Asociación con Rol ===
    rol_id = item.get("id_rol")
    if rol_id:
        metadata["rol_id"] = rol_id
        metadata["rol_nombre"] = mapa_roles.get(rol_id, "Desconocido")

    # === Riesgos asociados ===
    riesgos_ids = item.get("riesgos_asociados", []) or []
    if riesgos_ids:
        metadata["riesgos_asociados_ids"] = ", ".join(riesgos_ids)
        metadata["riesgos_asociados_nombres"] = ", ".join(
            [mapa_riesgos.get(rid, "Desconocido") for rid in riesgos_ids]
        )
        metadata["tiene_riesgos_asociados"] = True
    else:
        metadata["tiene_riesgos_asociados"] = False

    # === Controles críticos asociados ===
    controles_ids = item.get("controles_criticos_asociados", []) or []
    if controles_ids:
        metadata["controles_asociados_ids"] = ", ".join(controles_ids)
        metadata["controles_asociados_nombres"] = ", ".join(
            [mapa_controles.get(cid, "Desconocido") for cid in controles_ids]
        )
        metadata["tiene_controles_asociados"] = True
    else:
        metadata["tiene_controles_asociados"] = False

    # === Factores de exposición ===
    factores_ids = item.get("factores_de_exposicion", []) or []
    if factores_ids:
        metadata["factores_exposicion_ids"] = ", ".join(factores_ids)
        metadata["factores_exposicion_nombres"] = ", ".join(
            [mapa_factores.get(fid, "Desconocido") for fid in factores_ids]
        )
        metadata["tiene_factores_exposicion"] = True
    else:
        metadata["tiene_factores_exposicion"] = False

    # === Necesidades de EPP ===
    epp_ids = item.get("necesidades_epp", []) or []
    if epp_ids:
        metadata["epp_ids"] = ", ".join(epp_ids)
        metadata["epp_nombres"] = ", ".join(
            [mapa_epp.get(eid, "Desconocido") for eid in epp_ids]
        )
        metadata["tiene_epp"] = True
    else:
        metadata["tiene_epp"] = False

    # Área operacional
    metadata["area_operacional"] = item.get("area_operacional", "")

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_tareas.append(node)

print(f"✔️ Nodos de tareas procesados: {len(nodes_tareas)}")
if nodes_tareas:
    ejemplo = nodes_tareas[0]
    print("🔍 Ejemplo de nodo de tarea:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de tareas procesados: 96
🔍 Ejemplo de nodo de tarea:
ID: MRA_TA_001
Texto: Tarea: Operación de perforadoras rotativas en frentes de voladura. Operar perforadoras rotativas de gran tamaño (ej. DML, Pit Viper) para la creación de pozos de voladura. Esta tarea implica el posicionamiento preciso del mástil de perforación en puntos predefinidos por el plan de perforación, con una tolerancia de +/- 0.5 metros, sobre terrenos irregulares que pueden presentar pendientes de hasta 15 grados y condiciones de baja visibilidad por polvo o niebla. Además, el operador debe realizar inspecciones diarias del equipo y el mantenimiento básico en altura, como la lubricación de la broca y el cambio de herramientas de corte en la torre (a una altura de 3 a 5 metros), lo que requiere el uso de plataformas y escaleras integradas al equipo.
Metadatos: {'rol_id': 'MRA_ROL_01', 'rol_nombre': 'Operador de Perforadora', 'riesgos_asociados_ids': 'R01', 'riesgos_asociados_nombres': 'Caída desde Altura', 'ti

In [23]:
# 📌 Celda 27 - TA2 — Inserción segura de Tareas en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_tareas = [n.node_id for n in nodes_tareas]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_tareas,
    where=None,
    limit=999999
)
ids_tareas_chroma = set(chroma_check["ids"])
print(f"Tareas ya en Chroma: {len(ids_tareas_chroma)}/{len(ids_tareas)}")

# 3️⃣ Consultar en docstore (storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_tareas_docstore = {id_ for id_ in ids_tareas if id_ in ids_docstore}
print(f"Tareas ya en docstore: {len(ids_tareas_docstore)}/{len(ids_tareas)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_tareas if n.node_id not in ids_tareas_chroma]
nodos_faltan_docstore = [n for n in nodes_tareas if n.node_id not in ids_tareas_docstore]

print(f"Nodos de tareas a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de tareas a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Tareas insertadas en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Tareas insertadas en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Tareas ya en Chroma: 0/96
Tareas ya en docstore: 0/96
Nodos de tareas a insertar en Chroma: 96
Nodos de tareas a insertar en docstore: 96
Tareas insertadas en Chroma.
Tareas insertadas en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [25]:
# 📌 Celda 28 - TA3 — Verificación integral con Tareas incluidas

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo tareas)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$"
}

# 3️⃣ Conteo
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Tareas
ids_tareas_chroma = [id_ for id_ in ids_chroma if re.match(r"^[A-Z]{3}_TA_\d+$", id_)]
print(f"\nTotal de Tareas en Chroma: {len(ids_tareas_chroma)}")

if ids_tareas_chroma:
    sample_tareas = random.sample(ids_tareas_chroma, min(3, len(ids_tareas_chroma)))
    print("\nEjemplo específico de nodos de Tareas en Chroma:")

    for sid in sample_tareas:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 211

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Áreas Operacionales,5,5
7,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: TME_TA_004
Texto: Tarea: Calibración y reparación de sistemas de medición eléctrica. Realizar calibraciones, diagnósticos y reparaciones en sistemas de control y automatización (PLC, DCS), que implican la manipulación  ...
Metadatos: {'area_operacional': 'TME', 'controles_asociados_ids': 'R03CC01, R03CC02, R03CC03, R03CC04, R03CC05, R03CC06, R03CP01, R03CP02, R03CP03, R03CP04, R03CP05, R03CP06', 'controles_asociados_nombres': 'Procedimiento de Bloqueo y Etiquetado (LOTO) implementado y auditado, Mantenimiento preventivo y correctivo de instalaciones y equipos eléctricos, Uso obligatorio y verificación de EPP dieléctrico certificado, Demarcación y control de acceso a zonas de riesgo eléctrico, Capacitación y certificación de personal para trabajos eléctricos, Dispositivos de protección eléctrica (interruptores diferenciales, termomagnéticos) operativos, Identificación y aislamiento de fuentes de energía antes de intervenir, Inspección

In [26]:
# 📌 Celda 29 - RL1 — Crear nodos de Roles desde YAML con nombres de tareas incluidos

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de tareas para mapear IDs -> nombres ===
yaml_tareas = os.path.join(ONTOLOGY_DIR, "12_catalogo_tareas_v1.yaml")
with open(yaml_tareas, "r", encoding="utf-8") as f:
    tareas = yaml.safe_load(f)
mapa_tareas = {item["id"]: item.get("nombre_tarea", "") for item in tareas}

# === Cargar catálogo de roles ===
yaml_roles = os.path.join(ONTOLOGY_DIR, "10_catalogo_roles_v3.yaml")
with open(yaml_roles, "r", encoding="utf-8") as f:
    roles = yaml.safe_load(f)

nodes_roles = []
for item in roles:
    # Texto principal con prefijo explícito
    content = f"Rol: {item.get('nombre', '')}. {item.get('descripcion', '')}"
    metadata = {}

    # Campos simples
    metadata["area_operacional"] = item.get("area_operacional", "")
    metadata["clasificacion_funcional"] = item.get("clasificacion_funcional", "")
    metadata["rol_categoria"] = item.get("rol_categoria", "")

    # === Tareas asociadas (IDs + nombres) ===
    tareas_ids = item.get("tareas_asociadas", []) or []
    if tareas_ids:
        metadata["tareas_asociadas_ids"] = ", ".join(tareas_ids)
        metadata["tareas_asociadas_nombres"] = ", ".join(
            [mapa_tareas.get(tid, "Desconocida") for tid in tareas_ids]
        )
        metadata["tiene_tareas_asociadas"] = True
    else:
        metadata["tareas_asociadas_ids"] = ""
        metadata["tareas_asociadas_nombres"] = ""
        metadata["tiene_tareas_asociadas"] = False

    # Flag de área
    metadata["tiene_area_operacional"] = bool(item.get("area_operacional"))

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_roles.append(node)

print(f"✔️ Nodos de roles procesados: {len(nodes_roles)}")
if nodes_roles:
    ejemplo = nodes_roles[0]
    print("🔍 Ejemplo de nodo de rol:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de roles procesados: 45
🔍 Ejemplo de nodo de rol:
ID: MRA_ROL_01
Texto: Rol: Operador de Perforadora. Responsable de la operación segura y eficiente de equipos de perforación rotatoria para la creación de pozos de voladura en minería a cielo abierto. Incluye la inspección preoperacional, posicionamiento del equipo, perforación según diseño, control de parámetros de perforación y mantenimiento básico. Debe seguir estrictamente los procedimientos de seguridad, usar el EPP adecuado y estar certificado para trabajos en altura y manejo de equipos pesados. Su trabajo es crítico para la fragmentación eficiente de la roca.
Metadatos: {'area_operacional': 'MRA', 'clasificacion_funcional': 'Operativo', 'rol_categoria': 'Operador', 'tareas_asociadas_ids': 'MRA_TA_001, MRA_TA_002, MRA_TA_003', 'tareas_asociadas_nombres': 'Operación de perforadoras rotativas en frentes de voladura, Supervisión y mitigación de proyección de fragmentos durante perforación, Mantenimiento eléctrico de perforad

In [27]:
# 📌 Celda 30 - RL2 — Inserción segura de Roles en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_roles = [n.node_id for n in nodes_roles]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_roles,
    where=None,
    limit=999999
)
ids_roles_chroma = set(chroma_check["ids"])
print(f"Roles ya en Chroma: {len(ids_roles_chroma)}/{len(ids_roles)}")

# 3️⃣ Consultar en docstore (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_roles_docstore = {id_ for id_ in ids_roles if id_ in ids_docstore}
print(f"Roles ya en docstore: {len(ids_roles_docstore)}/{len(ids_roles)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_roles if n.node_id not in ids_roles_chroma]
nodos_faltan_docstore = [n for n in nodes_roles if n.node_id not in ids_roles_docstore]

print(f"Nodos de roles a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de roles a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Roles insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Roles insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Roles ya en Chroma: 0/45
Roles ya en docstore: 0/45
Nodos de roles a insertar en Chroma: 45
Nodos de roles a insertar en docstore: 45
Roles insertados en Chroma.
Roles insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [28]:
# 📌 Celda 31 - RL3 — Verificación integral con Roles incluidos y muestra específica de Roles

import re
import random
import pandas as pd

# 1️⃣ Obtener todos los registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo roles)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$"
}

# 3️⃣ Conteo
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 5️⃣ Muestreo específico de Roles
ids_roles_chroma = [id_ for id_ in ids_chroma if re.match(r"^[A-Z]{3}_ROL_\d+$", id_)]
print(f"\nTotal de Roles en Chroma: {len(ids_roles_chroma)}")

if ids_roles_chroma:
    sample_roles = random.sample(ids_roles_chroma, min(3, len(ids_roles_chroma)))
    print("\nEjemplo específico de nodos de Roles en Chroma:")

    for sid in sample_roles:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})



Total de IDs en Chroma: 256

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Áreas Operacionales,5,5
7,Tareas,96,96
8,Roles,45,45



Ejemplo aleatorio de nodos en Chroma (global):

ID: STM_ROL_03
Texto: Rol: Planificador de Mantenimiento. Elabora los programas de mantenimiento preventivo y correctivo para todos los sistemas de transporte mecanizado, incluyendo cintas, stackers, reclaimers y dumpers.  ...
Metadatos: {'area_operacional': 'STM', 'clasificacion_funcional': 'Supervisión', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'rol_categoria': 'Supervisor', 'tareas_asociadas_ids': 'STM_TA_005, STM_TA_006', 'tareas_asociadas_nombres': 'Recorridos de inspección por las secciones del sistema de transporte, Inspección de tableros eléctricos y puntos de conexión de potencia', 'tiene_area_operacional': True, 'tiene_tareas_asociadas': True}

ID: R02FD04
Texto: Factor de Degradación: Inexistencia de barreras inferiores/rodapiés. Falta de rodapiés o barreras de contención a nivel del suelo o en plataformas elevadas. Permite que herramientas, materiales o esco ...
Metadatos: {'aplicabilidad': 'Prevención d

In [29]:
# 📌 Celda 32 - FE1 — Crear nodos de Factores de Exposición con nombres de tareas, riesgos, roles y áreas

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
# Tareas
yaml_tareas = os.path.join(ONTOLOGY_DIR, "12_catalogo_tareas_v1.yaml")
with open(yaml_tareas, "r", encoding="utf-8") as f:
    tareas = yaml.safe_load(f)
mapa_tareas = {item["id"]: item.get("nombre_tarea", "") for item in tareas}

# Riesgos
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# Roles
yaml_roles = os.path.join(ONTOLOGY_DIR, "10_catalogo_roles_v3.yaml")
with open(yaml_roles, "r", encoding="utf-8") as f:
    roles = yaml.safe_load(f)
mapa_roles = {item["id"]: item.get("nombre", "") for item in roles}

# Áreas
yaml_areas = os.path.join(ONTOLOGY_DIR, "13_catalogo_areas_operacionales_v1.yaml")
with open(yaml_areas, "r", encoding="utf-8") as f:
    areas = yaml.safe_load(f)
mapa_areas = {item["id"]: item.get("nombre", "") for item in areas}

# === Cargar catálogo de factores de exposición ===
yaml_factores = os.path.join(ONTOLOGY_DIR, "11_catalogo_factores_exposicion_v3.yaml")
with open(yaml_factores, "r", encoding="utf-8") as f:
    factores = yaml.safe_load(f)

nodes_exposicion = []
for item in factores:
    # Texto principal con prefijo
    content = f"Factor de Exposición: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Tareas asociadas ===
    tareas_ids = item.get("tareas_asociadas", []) or []
    if tareas_ids:
        metadata["tareas_asociadas_ids"] = ", ".join(tareas_ids)
        metadata["tareas_asociadas_nombres"] = ", ".join(
            [mapa_tareas.get(tid, "Desconocida") for tid in tareas_ids]
        )
        metadata["tiene_tareas_asociadas"] = True
    else:
        metadata["tiene_tareas_asociadas"] = False

    # === Riesgos asociados ===
    riesgos_ids = item.get("riesgos_asociados", []) or []
    if riesgos_ids:
        metadata["riesgos_asociados_ids"] = ", ".join(riesgos_ids)
        metadata["riesgos_asociados_nombres"] = ", ".join(
            [mapa_riesgos.get(rid, "Desconocido") for rid in riesgos_ids]
        )
        metadata["tiene_riesgos_asociados"] = True
    else:
        metadata["tiene_riesgos_asociados"] = False

    # === Roles asociados ===
    roles_ids = item.get("roles_asociados", []) or []
    if roles_ids:
        metadata["roles_asociados_ids"] = ", ".join(roles_ids)
        metadata["roles_asociados_nombres"] = ", ".join(
            [mapa_roles.get(rid, "Desconocido") for rid in roles_ids]
        )
        metadata["tiene_roles_asociados"] = True
    else:
        metadata["tiene_roles_asociados"] = False

    # === Áreas asociadas ===
    areas_ids = item.get("areas_asociadas", []) or []
    if areas_ids:
        metadata["areas_asociadas_ids"] = ", ".join(areas_ids)
        metadata["areas_asociadas_nombres"] = ", ".join(
            [mapa_areas.get(aid, "Desconocida") for aid in areas_ids]
        )
        metadata["tiene_areas_asociadas"] = True
    else:
        metadata["tiene_areas_asociadas"] = False

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_exposicion.append(node)

print(f"✔️ Nodos de factores de exposición procesados: {len(nodes_exposicion)}")
if nodes_exposicion:
    ejemplo = nodes_exposicion[0]
    print("🔍 Ejemplo de nodo de factor de exposición:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de factores de exposición procesados: 211
🔍 Ejemplo de nodo de factor de exposición:
ID: FE_001
Texto: Factor de Exposición: Trabajo en Altura. Realización de tareas sobre plataformas elevadas, estructuras metálicas o zonas sin protección perimetral. Este factor implica exposición directa a riesgos de caída por pérdida de equilibrio, tropiezos o ausencia de puntos de anclaje adecuados. Las tareas suelen involucrar inspecciones, mantenimientos o montaje de componentes, requiriendo sistemas anticaída certificados, líneas de vida, barandas o andamios seguros.
Metadatos: {'tareas_asociadas_ids': 'MRA_TA_001', 'tareas_asociadas_nombres': 'Operación de perforadoras rotativas en frentes de voladura', 'tiene_tareas_asociadas': True, 'riesgos_asociados_ids': 'R01', 'riesgos_asociados_nombres': 'Caída desde Altura', 'tiene_riesgos_asociados': True, 'roles_asociados_ids': 'MRA_ROL_01', 'roles_asociados_nombres': 'Operador de Perforadora', 'tiene_roles_asociados': True, 'areas_asociadas_i

In [30]:
# 📌 Celda 33 - FE2 — Inserción segura de Factores de Exposición en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_exposicion = [n.node_id for n in nodes_exposicion]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_exposicion,
    where=None,
    limit=999999
)
ids_exposicion_chroma = set(chroma_check["ids"])
print(f"Factores de Exposición ya en Chroma: {len(ids_exposicion_chroma)}/{len(ids_exposicion)}")

# 3️⃣ Consultar en docstore (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_exposicion_docstore = {id_ for id_ in ids_exposicion if id_ in ids_docstore}
print(f"Factores de Exposición ya en docstore: {len(ids_exposicion_docstore)}/{len(ids_exposicion)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_exposicion if n.node_id not in ids_exposicion_chroma]
nodos_faltan_docstore = [n for n in nodes_exposicion if n.node_id not in ids_exposicion_docstore]

print(f"Nodos de factores de exposición a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de factores de exposición a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Factores de Exposición insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Factores de Exposición insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Factores de Exposición ya en Chroma: 0/211
Factores de Exposición ya en docstore: 0/211
Nodos de factores de exposición a insertar en Chroma: 211
Nodos de factores de exposición a insertar en docstore: 211
Factores de Exposición insertados en Chroma.
Factores de Exposición insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [31]:
# 📌 Celda 34 - FE3 — Verificación integral con Factores de Exposición incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo factores de exposición)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$"
}

# 3️⃣ Conteo
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Factores de Exposición
ids_exposicion_chroma = [id_ for id_ in ids_chroma if re.match(r"^FE_\d+$", id_)]
print(f"\nTotal de Factores de Exposición en Chroma: {len(ids_exposicion_chroma)}")

if ids_exposicion_chroma:
    sample_exposicion = random.sample(ids_exposicion_chroma, min(3, len(ids_exposicion_chroma)))
    print("\nEjemplo específico de nodos de Factores de Exposición en Chroma:")

    for sid in sample_exposicion:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 467

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Tareas,96,96
9,Roles,45,45



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_060
Texto: Factor de Exposición: Tránsito Cercano a Equipos con Puntos Ciegos. Dirigir el tránsito a pie cerca de equipos pesados como palas, camiones o perforadoras implica riesgo de impacto si el operador no v ...
Metadatos: {'areas_asociadas_ids': 'MRA', 'areas_asociadas_nombres': 'Mina Rajo Abierto', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R01', 'riesgos_asociados_nombres': 'Caída desde Altura', 'roles_asociados_ids': 'MRA_ROL_08', 'roles_asociados_nombres': 'Guía/Controlador de Tráfico', 'tareas_asociadas_ids': 'MRA_TA_022', 'tareas_asociadas_nombres': 'Dirección y control de tráfico de equipos en zonas de carguío', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: PLC_TA_004
Texto: Tarea: Supervisión de tableros de control de molinos y sistemas de accionamiento. Supervisar los tableros de control

In [32]:
# 📌 Celda 35 - EPP1 — Crear nodos de EPP desde YAML con prefijo y metadatos estandarizados

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de EPP ===
yaml_epp = os.path.join(ONTOLOGY_DIR, "14_catalogo_epp_v3.yaml")
with open(yaml_epp, "r", encoding="utf-8") as f:
    epp_items = yaml.safe_load(f)

nodes_epp = []
for item in epp_items:
    # Texto principal con prefijo
    content = f"EPP: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # Campos simples
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")

    # Listas convertidas a string
    tags = item.get("tags", [])
    metadata["tags"] = ", ".join(tags) if isinstance(tags, list) else str(tags)

    registro = item.get("registro_aplicable_a", [])
    metadata["registro_aplicable_a"] = ", ".join(registro) if isinstance(registro, list) else str(registro)

    # Flags booleanos
    metadata["tiene_tags"] = bool(item.get("tags"))
    metadata["tiene_registro_aplicable_a"] = bool(item.get("registro_aplicable_a"))

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_epp.append(node)

print(f"✔️ Nodos de EPP procesados: {len(nodes_epp)}")
if nodes_epp:
    ejemplo = nodes_epp[0]
    print("🔍 Ejemplo de nodo de EPP:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de EPP procesados: 66
🔍 Ejemplo de nodo de EPP:
ID: EPP_001
Texto: EPP: Casco de seguridad. Casco de seguridad estándar, para protección de la cabeza contra impactos y objetos que caen.
Metadatos: {'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Uso de equipo de protección personal', 'tags': 'protección_cabeza, impacto, caída_objetos, seguridad_craneal, barbuquejo, norma_EN_397, hard_hat', 'registro_aplicable_a': 'incidente, observacion', 'tiene_tags': True, 'tiene_registro_aplicable_a': True}


In [33]:
# 📌 Celda 36 - EPP2 — Inserción segura de EPP en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_epp = [n.node_id for n in nodes_epp]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_epp,
    where=None,
    limit=999999
)
ids_epp_chroma = set(chroma_check["ids"])
print(f"EPP ya en Chroma: {len(ids_epp_chroma)}/{len(ids_epp)}")

# 3️⃣ Consultar en docstore (usamos storage_context.docstore)
ids_docstore = set(storage_context.docstore.docs.keys())
ids_epp_docstore = {id_ for id_ in ids_epp if id_ in ids_docstore}
print(f"EPP ya en docstore: {len(ids_epp_docstore)}/{len(ids_epp)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_epp if n.node_id not in ids_epp_chroma]
nodos_faltan_docstore = [n for n in nodes_epp if n.node_id not in ids_epp_docstore]

print(f"Nodos de EPP a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de EPP a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("EPP insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("EPP insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


EPP ya en Chroma: 0/66
EPP ya en docstore: 0/66
Nodos de EPP a insertar en Chroma: 66
Nodos de EPP a insertar en docstore: 66
EPP insertados en Chroma.
EPP insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [34]:
# 📌 Celda 37 - EPP3 — Verificación integral con EPP incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo EPP)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$"
}

# 3️⃣ Conteo
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")

    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de EPP
ids_epp_chroma = [id_ for id_ in ids_chroma if re.match(r"^EPP_\d+$", id_)]
print(f"\nTotal de EPP en Chroma: {len(ids_epp_chroma)}")

if ids_epp_chroma:
    sample_epp = random.sample(ids_epp_chroma, min(3, len(ids_epp_chroma)))
    print("\nEjemplo específico de nodos de EPP en Chroma:")

    for sid in sample_epp:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 533

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_157
Texto: Factor de Exposición: Aplastamiento por Cinta o Equipos No Asegurados. El posicionamiento de equipos sin fijación o interferencia con la cinta en movimiento puede generar atrapamientos graves durante  ...
Metadatos: {'areas_asociadas_ids': 'STM', 'areas_asociadas_nombres': 'Sistema de Transporte Mecanizado', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R01', 'riesgos_asociados_nombres': 'Caída desde Altura', 'roles_asociados_ids': 'STM_ROL_06', 'roles_asociados_nombres': 'Empalmador de Cinta Transportadora', 'tareas_asociadas_ids': 'STM_TA_011', 'tareas_asociadas_nombres': 'Empalme y reparación de cintas transportadoras en terreno', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: EPP_025
Texto: EPP: Traje de enfriamiento (cooling vest). Chaleco o traje diseñado para mantener la temperatura corp

In [7]:
# 📌 Celda 38 - P1 — Crear nodos de Peligros con nombres de riesgos y áreas + alias

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
# Riesgos
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# Áreas
yaml_areas = os.path.join(ONTOLOGY_DIR, "13_catalogo_areas_operacionales_v1.yaml")
with open(yaml_areas, "r", encoding="utf-8") as f:
    areas = yaml.safe_load(f)
mapa_areas = {item["id"]: item.get("nombre", "") for item in areas}

# === Cargar catálogo de Peligros ===
yaml_peligros = os.path.join(ONTOLOGY_DIR, "17_catalogo_peligros_detectados_v5.yaml")
with open(yaml_peligros, "r", encoding="utf-8") as f:
    peligros = yaml.safe_load(f)

nodes_peligros = []
for item in peligros:
    # Texto principal con prefijo
    content = f"Peligro: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Riesgo principal ===
    riesgo_id = item.get("riesgo_principal_id", "")
    if riesgo_id:
        metadata["riesgo_principal_id"] = riesgo_id
        metadata["riesgo_principal_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")
        metadata["tiene_riesgo_principal"] = True
    else:
        metadata["tiene_riesgo_principal"] = False
    # Alias de riesgo
    metadata["alias_riesgo"] = item.get("riesgo_principal", "")

    # === Área asociada ===
    area_id = item.get("area_id", "")
    if isinstance(area_id, list):
        metadata["area_id"] = ", ".join(area_id)
        metadata["area_nombre"] = ", ".join([mapa_areas.get(aid, "Desconocida") for aid in area_id])
        metadata["tiene_area"] = True
    elif area_id:
        metadata["area_id"] = area_id
        metadata["area_nombre"] = mapa_areas.get(area_id, "Desconocida")
        metadata["tiene_area"] = True
    else:
        metadata["tiene_area"] = False
    # Alias de área
    metadata["alias_area"] = item.get("area", "")

    # === Campos adicionales ===
    metadata["clasificacion"] = item.get("clasificacion", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Crear nodo enriquecido
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_peligros.append(node)

print(f"✔️ Nodos de Peligros procesados: {len(nodes_peligros)}")
if nodes_peligros:
    ejemplo = nodes_peligros[0]
    print("🔍 Ejemplo de nodo de Peligros:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Peligros procesados: 129
🔍 Ejemplo de nodo de Peligros:
ID: P001
Texto: Peligro: Herramientas sueltas en altura. Presencia de herramientas no aseguradas sobre plataformas elevadas, con riesgo de caída hacia personal u otros niveles.
Metadatos: {'riesgo_principal_id': 'R02', 'riesgo_principal_nombre': 'Caída de Objetos', 'tiene_riesgo_principal': True, 'alias_riesgo': 'Caída de objetos', 'area_id': 'PLC', 'area_nombre': 'Planta Concentradora', 'tiene_area': True, 'alias_area': 'Planta Concentradora', 'clasificacion': 'Condición peligrosa', 'aplicabilidad': 'Detección de peligros', 'registro_aplicable_a': 'observacion', 'tags': 'caída_objetos, objetos_sueltos, altura, plataforma, seguridad_operacional'}


In [8]:
# 📌 Celda 39 - P2 — Inserción segura de Peligros en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_peligros = [n.node_id for n in nodes_peligros]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_peligros,
    where=None,
    limit=999999
)
ids_peligros_chroma = set(chroma_check["ids"])
print(f"Peligros ya en Chroma: {len(ids_peligros_chroma)}/{len(ids_peligros)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_peligros_docstore = {id_ for id_ in ids_peligros if id_ in ids_docstore}
print(f"Peligros ya en docstore: {len(ids_peligros_docstore)}/{len(ids_peligros)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_peligros if n.node_id not in ids_peligros_chroma]
nodos_faltan_docstore = [n for n in nodes_peligros if n.node_id not in ids_peligros_docstore]

print(f"Nodos de Peligros a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Peligros a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Peligros insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Peligros insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Peligros ya en Chroma: 0/129
Peligros ya en docstore: 0/129
Nodos de Peligros a insertar en Chroma: 129
Nodos de Peligros a insertar en docstore: 129
Peligros insertados en Chroma.
Peligros insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [9]:
# 📌 Celda 40 - P3 — Verificación integral con Peligros incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo Peligros)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Peligros
ids_peligros_chroma = [id_ for id_ in ids_chroma if re.match(r"^P\d+$", id_)]
print(f"\nTotal de Peligros en Chroma: {len(ids_peligros_chroma)}")

if ids_peligros_chroma:
    sample_peligros = random.sample(ids_peligros_chroma, min(3, len(ids_peligros_chroma)))
    print("\nEjemplo específico de nodos de Peligros en Chroma:")
    for sid in sample_peligros:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 662

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: P042
Texto: Peligro: Acumulación de material particulado respirable. Concentraciones elevadas de polvo fino o partículas en suspensión en el ambiente de trabajo. 
Metadatos: {'alias_area': 'Mina Subterránea / Zonas de Chancado', 'alias_riesgo': 'Enfermedades respiratorias', 'aplicabilidad': 'Detección de peligros', 'area_id': 'CHP', 'area_nombre': 'Planta de Chancado Primario', 'clasificacion': 'Condición peligrosa', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'registro_aplicable_a': 'observacion', 'riesgo_principal_id': 'NA', 'riesgo_principal_nombre': 'Desconocido', 'tags': 'material_particulado, polvo_respirable, silicosis, neumoconiosis, enfermedades_respiratorias, calidad_aire, salud_ocupacional', 'tiene_area': True, 'tiene_riesgo_principal': True}

ID: FE_140
Texto: Factor de Exposición: Trabajo en Altura sobre Pasarelas Contaminadas. Ejecutar tareas sobre pasarelas con presencia de polvo, vibración y derrame

In [10]:
# 📌 Celda 41 - EVT1 — Crear nodos de Eventos Principales con riesgos y áreas + alias

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogos de referencia ===
# Riesgos
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# Áreas
yaml_areas = os.path.join(ONTOLOGY_DIR, "13_catalogo_areas_operacionales_v1.yaml")
with open(yaml_areas, "r", encoding="utf-8") as f:
    areas = yaml.safe_load(f)
mapa_areas = {item["id"]: item.get("nombre", "") for item in areas}

# === Cargar catálogo de Eventos Principales ===
yaml_eventos = os.path.join(ONTOLOGY_DIR, "18_catalogo_eventos_principales_v2.yaml")
with open(yaml_eventos, "r", encoding="utf-8") as f:
    eventos = yaml.safe_load(f)

nodes_eventos = []
for item in eventos:
    # Texto principal
    content = f"Evento Principal: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Riesgo asociado ===
    riesgo_id = item.get("riesgo_asociado", "")
    if riesgo_id:
        metadata["riesgo_asociado_id"] = riesgo_id
        metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")
        metadata["tiene_riesgo_asociado"] = True
    else:
        metadata["tiene_riesgo_asociado"] = False
    metadata["alias_riesgo"] = riesgo_id  # alias directo del YAML

    # === Área asociada ===
    area_txt = item.get("area", "")
    if area_txt == "Todas las Áreas Operacionales":
        area_ids = ["MRA", "CHP", "TME", "STM", "PLC"]
        metadata["area_id"] = ", ".join(area_ids)
        metadata["area_nombre"] = ", ".join([mapa_areas.get(aid, "Desconocida") for aid in area_ids])
        metadata["tiene_area"] = True
    else:
        metadata["area_id"] = ""
        metadata["area_nombre"] = ""
        metadata["tiene_area"] = False
    metadata["alias_area"] = area_txt

    # === Otros campos ===
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_eventos.append(node)

print(f"✔️ Nodos de Eventos Principales procesados: {len(nodes_eventos)}")
if nodes_eventos:
    ejemplo = nodes_eventos[0]
    print("🔍 Ejemplo de nodo de Evento Principal:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Eventos Principales procesados: 10
🔍 Ejemplo de nodo de Evento Principal:
ID: EVT01
Texto: Evento Principal: Pérdida de equilibrio y caída desde altura. Este evento principal describe la situación en la que una persona pierde el control de su posición y desciende de forma incontrolada desde un nivel superior a uno inferior. Generalmente ocurre en trabajos en altura, al transitar por superficies elevadas o cerca de aberturas desprotegidas. La caída puede ser provocada por fallas en superficies, equipos de protección personal, condiciones climáticas o errores humanos. Es el punto de liberación de energía antes de que se manifiesten las consecuencias de una caída.
Metadatos: {'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'tiene_riesgo_asociado': True, 'alias_riesgo': 'R01', 'area_id': 'MRA, CHP, TME, STM, PLC', 'area_nombre': 'Mina Rajo Abierto, Planta de Chancado Primario, Taller de Mantenimiento Eléctrico, Sistema de Transporte Mecanizado, Plan

In [11]:
# 📌 Celda 42 - EVT2 — Inserción segura de Eventos Principales en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_eventos = [n.node_id for n in nodes_eventos]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_eventos,
    where=None,
    limit=999999
)
ids_eventos_chroma = set(chroma_check["ids"])
print(f"Eventos ya en Chroma: {len(ids_eventos_chroma)}/{len(ids_eventos)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_eventos_docstore = {id_ for id_ in ids_eventos if id_ in ids_docstore}
print(f"Eventos ya en docstore: {len(ids_eventos_docstore)}/{len(ids_eventos)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_eventos if n.node_id not in ids_eventos_chroma]
nodos_faltan_docstore = [n for n in nodes_eventos if n.node_id not in ids_eventos_docstore]

print(f"Nodos de Eventos a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Eventos a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Eventos insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Eventos insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Eventos ya en Chroma: 0/10
Eventos ya en docstore: 0/10
Nodos de Eventos a insertar en Chroma: 10
Nodos de Eventos a insertar en docstore: 10
Eventos insertados en Chroma.
Eventos insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [12]:
# 📌 Celda 43 - EVT3 — Verificación integral con Eventos incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)

ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo Eventos)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Eventos
ids_eventos_chroma = [id_ for id_ in ids_chroma if re.match(r"^EVT\d+$", id_)]
print(f"\nTotal de Eventos en Chroma: {len(ids_eventos_chroma)}")

if ids_eventos_chroma:
    sample_eventos = random.sample(ids_eventos_chroma, min(3, len(ids_eventos_chroma)))
    print("\nEjemplo específico de nodos de Eventos en Chroma:")
    for sid in sample_eventos:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 672

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: CHP_ROL_06
Texto: Rol: Jefe de Turno de Chancado. Lidera, organiza y supervisa las operaciones de un turno completo en la planta de chancado, siendo responsable de la seguridad del personal y la eficiencia de la produc ...
Metadatos: {'area_operacional': 'CHP', 'clasificacion_funcional': 'Supervisión', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'rol_categoria': 'Supervisor', 'tareas_asociadas_ids': 'CHP_TA_011, CHP_TA_012', 'tareas_asociadas_nombres': 'Muestreo y mapeo geológico en frentes de la planta de chancado, Supervisión de condiciones geomecánicas del material chancado', 'tiene_area_operacional': True, 'tiene_tareas_asociadas': True}

ID: P005
Texto: Peligro: Acumulación de polvo en vías de tránsito. Presencia excesiva de polvo mineral en accesos peatonales o vehiculares, reduciendo visibilidad y creando riesgo de resbalones. 
Metadatos: {'alias_area': 'Vías de Tránsito de Mina', 'alias_riesgo': 'Resbalones

In [13]:
# 📌 Celda 44 - TI1 — Crear nodos de Tipos de Incidente

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de Tipos de Incidente ===
yaml_incidentes = os.path.join(ONTOLOGY_DIR, "25_tipos_incidentes_v1.yaml")
with open(yaml_incidentes, "r", encoding="utf-8") as f:
    tipos_incidentes = yaml.safe_load(f)

nodes_incidentes = []
for item in tipos_incidentes:
    # Texto principal
    content = f"Tipo de Incidente: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos específicos ===
    metadata["formato_analisis_id"] = item.get("formato_analisis_id", "")
    metadata["metodo_analisis_nombre"] = item.get("metodo_analisis_nombre", "")
    metadata["requiere_control_critico"] = item.get("requiere_control_critico", False)
    metadata["asociacion_bowtie"] = item.get("asociacion_bowtie", False)
    metadata["campos_obligatorios_reporte"] = ", ".join(item.get("campos_obligatorios_reporte", []))
    metadata["ejemplos_consecuencias"] = "; ".join(item.get("ejemplos_consecuencias", []))

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_incidentes.append(node)

print(f"✔️ Nodos de Tipos de Incidente procesados: {len(nodes_incidentes)}")
if nodes_incidentes:
    ejemplo = nodes_incidentes[0]
    print("🔍 Ejemplo de nodo de Tipo de Incidente:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Tipos de Incidente procesados: 4
🔍 Ejemplo de nodo de Tipo de Incidente:
ID: IMAY
Texto: Tipo de Incidente: Incidente Mayor. Evento con consecuencias severas como lesiones graves, incapacidad permanente o fatalidad. Requiere una investigación exhaustiva para identificar todas las causas y factores contribuyentes.
Metadatos: {'formato_analisis_id': 'ICAM', 'metodo_analisis_nombre': 'Incident Cause Analysis Method (ICAM)', 'requiere_control_critico': True, 'asociacion_bowtie': True, 'campos_obligatorios_reporte': 'id_incidente, fecha, hora, area_trabajo, actividad_involucrada, riesgo_principal_asociado, evento_principal_ocurrido, descripcion_incidente, consecuencias_reportadas, severidad_estimada, factores_contributivos_icam, acciones_correctivas_implementadas', 'ejemplos_consecuencias': 'Lesiones con hospitalización o amputación; Fatalidad; Daños estructurales mayores; Paralización prolongada de operaciones'}


In [14]:
# 📌 Celda 45 - TI2 — Inserción segura de Tipos de Incidente en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_incidentes = [n.node_id for n in nodes_incidentes]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_incidentes,
    where=None,
    limit=999999
)
ids_incidentes_chroma = set(chroma_check["ids"])
print(f"Tipos de Incidente ya en Chroma: {len(ids_incidentes_chroma)}/{len(ids_incidentes)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_incidentes_docstore = {id_ for id_ in ids_incidentes if id_ in ids_docstore}
print(f"Tipos de Incidente ya en docstore: {len(ids_incidentes_docstore)}/{len(ids_incidentes)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_incidentes if n.node_id not in ids_incidentes_chroma]
nodos_faltan_docstore = [n for n in nodes_incidentes if n.node_id not in ids_incidentes_docstore]

print(f"Nodos de Tipos de Incidente a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Tipos de Incidente a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Tipos de Incidente insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Tipos de Incidente insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Tipos de Incidente ya en Chroma: 0/4
Tipos de Incidente ya en docstore: 0/4
Nodos de Tipos de Incidente a insertar en Chroma: 4
Nodos de Tipos de Incidente a insertar en docstore: 4
Tipos de Incidente insertados en Chroma.
Tipos de Incidente insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [15]:
# 📌 Celda 46 - TI3 — Verificación integral con Tipos de Incidente incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo Tipos de Incidente)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$"  # IDs del catálogo
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Tipos de Incidente
ids_incidentes_chroma = [id_ for id_ in ids_chroma if re.match(r"^(IMAY|IMEN|NMIS|HZRD)$", id_)]
print(f"\nTotal de Tipos de Incidente en Chroma: {len(ids_incidentes_chroma)}")

if ids_incidentes_chroma:
    sample_incidentes = random.sample(ids_incidentes_chroma, min(3, len(ids_incidentes_chroma)))
    print("\nEjemplo específico de nodos de Tipos de Incidente en Chroma:")
    for sid in sample_incidentes:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 676

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_129
Texto: Factor de Exposición: Caída de Carga sin Elementos de Transporte. Transportar componentes eléctricos pesados sin carros o grúas expone a caídas por deslizamiento, mala postura o tropiezos durante el t ...
Metadatos: {'areas_asociadas_ids': 'TME', 'areas_asociadas_nombres': 'Taller de Mantenimiento Eléctrico', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R02', 'riesgos_asociados_nombres': 'Caída de Objetos', 'roles_asociados_ids': 'TME_ROL_07', 'roles_asociados_nombres': 'Encargado de Bodega de Repuestos Eléctricos', 'tareas_asociadas_ids': 'TME_TA_014', 'tareas_asociadas_nombres': 'Manipulación y almacenamiento seguro de componentes eléctricos pesados', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: FE_124
Texto: Factor de Exposición: Aplastamiento por Piezas Voluminosas. Manipular componentes

In [16]:
# 📌 Celda 47 - TO1 — Crear nodos de Tipos de Observación

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de Tipos de Observación ===
yaml_observaciones = os.path.join(ONTOLOGY_DIR, "26_tipos_observaciones_v2.yaml")
with open(yaml_observaciones, "r", encoding="utf-8") as f:
    tipos_observaciones = yaml.safe_load(f)

nodes_observaciones = []
for item in tipos_observaciones:
    # Texto principal
    content = f"Tipo de Observación: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos específicos ===
    metadata["requiere_control_critico"] = item.get("requiere_control_critico", False)
    metadata["riesgo_opcional"] = item.get("riesgo_opcional", False)
    metadata["observacion_valorativa"] = item.get("observacion_valorativa", False)
    metadata["asociacion_bowtie"] = item.get("asociacion_bowtie", False)
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_observaciones.append(node)

print(f"✔️ Nodos de Tipos de Observación procesados: {len(nodes_observaciones)}")
if nodes_observaciones:
    ejemplo = nodes_observaciones[0]
    print("🔍 Ejemplo de nodo de Tipo de Observación:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Tipos de Observación procesados: 2
🔍 Ejemplo de nodo de Tipo de Observación:
ID: OPG
Texto: Tipo de Observación: Observación Preventiva General. Observación de seguridad que documenta **comportamientos, condiciones o actos no seguros** que pueden dar lugar a un incidente si no se corrigen. Estas observaciones, aunque no requieren una consecuencia inmediata, ayudan a identificar tendencias y a prevenir riesgos antes de que se materialicen. No tienen que estar directamente asociadas a un riesgo pre-identificado.
Metadatos: {'requiere_control_critico': False, 'riesgo_opcional': True, 'observacion_valorativa': True, 'asociacion_bowtie': False, 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Clasificación de observaciones', 'registro_aplicable_a': 'observacion', 'tags': 'observación preventiva, condiciones inseguras, actos peligrosos, documentación de campo, análisis de tendencia'}


In [17]:
# 📌 Celda 48 - TO2 — Inserción segura de Tipos de Observación en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_observaciones = [n.node_id for n in nodes_observaciones]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_observaciones,
    where=None,
    limit=999999
)
ids_observaciones_chroma = set(chroma_check["ids"])
print(f"Tipos de Observación ya en Chroma: {len(ids_observaciones_chroma)}/{len(ids_observaciones)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_observaciones_docstore = {id_ for id_ in ids_observaciones if id_ in ids_docstore}
print(f"Tipos de Observación ya en docstore: {len(ids_observaciones_docstore)}/{len(ids_observaciones)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_observaciones if n.node_id not in ids_observaciones_chroma]
nodos_faltan_docstore = [n for n in nodes_observaciones if n.node_id not in ids_observaciones_docstore]

print(f"Nodos de Tipos de Observación a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Tipos de Observación a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Tipos de Observación insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Tipos de Observación insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Tipos de Observación ya en Chroma: 0/2
Tipos de Observación ya en docstore: 0/2
Nodos de Tipos de Observación a insertar en Chroma: 2
Nodos de Tipos de Observación a insertar en docstore: 2
Tipos de Observación insertados en Chroma.
Tipos de Observación insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [18]:
# 📌 Celda 49 - TO3 — Verificación integral con Tipos de Observación incluidos

import re
import random
import pandas as pd

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Definir patrones por categoría (incluyendo Tipos de Observación)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$"
}

# 3️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 4️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})

# 5️⃣ Muestreo específico de Tipos de Observación
ids_obs_chroma = [id_ for id_ in ids_chroma if re.match(r"^(OPG|OCC)$", id_)]
print(f"\nTotal de Tipos de Observación en Chroma: {len(ids_obs_chroma)}")

if ids_obs_chroma:
    sample_obs = random.sample(ids_obs_chroma, min(2, len(ids_obs_chroma)))
    print("\nEjemplo específico de nodos de Tipos de Observación en Chroma:")
    for sid in sample_obs:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith("_")})


Total de IDs en Chroma: 678

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_079
Texto: Factor de Exposición: Cableado Expuesto o Improvisado. La presencia de cables dañados, sin canalización o con tendido improvisado en salas técnicas puede provocar tropiezos, contactos accidentales o c ...
Metadatos: {'areas_asociadas_ids': 'CHP', 'areas_asociadas_nombres': 'Planta de Chancado Primario', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R03', 'riesgos_asociados_nombres': 'Contacto con Energía Eléctrica', 'roles_asociados_ids': 'CHP_ROL_04', 'roles_asociados_nombres': 'Ayudante de Planta', 'tareas_asociadas_ids': 'CHP_TA_009', 'tareas_asociadas_nombres': 'Monitoreo de sistemas de control de planta de chancado primario', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: FE_132
Texto: Factor de Exposición: Omisión de Distancias de Seguridad en Áreas Eléctricas. Ingresar o trabajar sin res

In [19]:
# 📌 Celda 50 - FA1 — Crear nodos de Formatos de Análisis

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de Formatos de Análisis ===
yaml_formatos = os.path.join(ONTOLOGY_DIR, "21_formatos_analisis_v3.yaml")
with open(yaml_formatos, "r", encoding="utf-8") as f:
    formatos = yaml.safe_load(f)

nodes_formatos = []
for item in formatos:
    # Texto principal
    content = f"Formato de Análisis: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos específicos ===
    metadata["aplicabilidad"] = ", ".join(item.get("aplicabilidad", []))
    metadata["campos_principales"] = ", ".join(item.get("campos_principales", []))
    metadata["area"] = item.get("area", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_formatos.append(node)

print(f"✔️ Nodos de Formatos de Análisis procesados: {len(nodes_formatos)}")
if nodes_formatos:
    ejemplo = nodes_formatos[0]
    print("🔍 Ejemplo de nodo de Formato de Análisis:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Formatos de Análisis procesados: 3
🔍 Ejemplo de nodo de Formato de Análisis:
ID: ICAM
Texto: Formato de Análisis: Incident Cause Analysis Method. Método estructurado de análisis de incidentes que examina factores humanos, organizacionales y del entorno. Identifica causas raíz en incidentes mayores y facilita la implementación de mejoras sistémicas. ICAM es ideal para incidentes de alta severidad, donde se requiere un entendimiento profundo de las interacciones complejas de los factores contribuyentes para desarrollar acciones correctivas robustas.
Metadatos: {'aplicabilidad': 'Incidente Mayor (IMAY)', 'campos_principales': 'descripcion_incidente, lesiones_daños, factores_organizacionales, factores_humanos, factores_tarea, factores_entorno, clasificacion_icam, acciones_correctivas', 'area': 'Todas las Áreas Operacionales', 'registro_aplicable_a': 'incidente', 'tags': 'icam, causas raíz, análisis estructurado, incidentes mayores, metodología causal, investigación exhaustiva, 

In [20]:
# 📌 Celda 51 - FA2 — Inserción segura de Formatos de Análisis en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_formatos = [n.node_id for n in nodes_formatos]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_formatos,
    where=None,
    limit=999999
)
ids_formatos_chroma = set(chroma_check["ids"])
print(f"Formatos de Análisis ya en Chroma: {len(ids_formatos_chroma)}/{len(ids_formatos)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_formatos_docstore = {id_ for id_ in ids_formatos if id_ in ids_docstore}
print(f"Formatos de Análisis ya en docstore: {len(ids_formatos_docstore)}/{len(ids_formatos)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_formatos if n.node_id not in ids_formatos_chroma]
nodos_faltan_docstore = [n for n in nodes_formatos if n.node_id not in ids_formatos_docstore]

print(f"Nodos de Formatos de Análisis a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Formatos de Análisis a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Formatos de Análisis insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Formatos de Análisis insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Formatos de Análisis ya en Chroma: 0/3
Formatos de Análisis ya en docstore: 0/3
Nodos de Formatos de Análisis a insertar en Chroma: 3
Nodos de Formatos de Análisis a insertar en docstore: 3
Formatos de Análisis insertados en Chroma.
Formatos de Análisis insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [22]:
# 📌 Celda 52 - FA3 — Verificación integral con Formatos de Análisis incluidos
# 📌 Celda 52 - FA3 — Verificación integral dinámica con Formatos de Análisis incluidos

import re
import random
import pandas as pd
import yaml

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Cargar IDs de Formatos de Análisis desde el YAML
yaml_formatos = os.path.join(ONTOLOGY_DIR, "21_formatos_analisis_v3.yaml")
with open(yaml_formatos, "r", encoding="utf-8") as f:
    formatos_yaml = yaml.safe_load(f)
ids_formatos_yaml = [item["id"] for item in formatos_yaml]
print(f"IDs esperados de Formatos de Análisis (YAML): {ids_formatos_yaml}")

# 3️⃣ Definir patrones por categoría (genéricos + dinámico para Formatos)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$"
}

# 4️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

# Agregar conteo dinámico de Formatos de Análisis
ids_formatos_chroma = [id_ for id_ in ids_chroma if id_ in ids_formatos_yaml]
ids_formatos_docstore = [id_ for id_ in ids_docstore if id_ in ids_formatos_yaml]
resumen.append({
    "Categoría": "Formatos de Análisis",
    "Chroma": len(ids_formatos_chroma),
    "Docstore": len(ids_formatos_docstore),
})

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 5️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 6️⃣ Muestreo específico de Formatos de Análisis
print(f"\nTotal de Formatos de Análisis en Chroma: {len(ids_formatos_chroma)}")
if ids_formatos_chroma:
    sample_formatos = random.sample(ids_formatos_chroma, min(3, len(ids_formatos_chroma)))
    print("\nEjemplo específico de nodos de Formatos de Análisis en Chroma:")
    for sid in sample_formatos:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})



Total de IDs en Chroma: 681
IDs esperados de Formatos de Análisis (YAML): ['ICAM', '5WHYS', 'OBS']

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: EPP_040
Texto: EPP: Traje de protección solar. Ropa ligera de manga larga y pantalón largo con tejido especializado que bloquea los rayos UV, para protección extendida en trabajos al sol. 
Metadatos: {'aplicabilidad': 'Uso de equipo de protección personal', 'area': 'Todas las Áreas Operacionales', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'protección_cuerpo, protección_UV, exposición_solar_intensa, trabajos_al_aire_libre, radiación_solar, ropa_ligera', 'tiene_registro_aplicable_a': True, 'tiene_tags': True}

ID: EPP_057
Texto: EPP: Arnés de suspensión (para trabajos prolongados). Arnés diseñado para la comodidad en trabajos que implican suspensión prolongada, con acolchado extra y puntos de soporte. 
Metadatos: {'aplicabilidad': 'Uso de equipo de protección personal', 'area': 'Todas las Áreas Operacionales', 'doc_id': 'None', 'document_id': 'None', 'ref_do

In [23]:
# 📌 Celda 53 - ICAM1 — Crear nodos de Factores ICAM

import yaml
from llama_index.core.schema import TextNode

# === Cargar catálogo de Factores ICAM ===
yaml_icam = os.path.join(ONTOLOGY_DIR, "22_factores_icam_v3.yaml")
with open(yaml_icam, "r", encoding="utf-8") as f:
    factores_icam = yaml.safe_load(f)

nodes_icam = []
for item in factores_icam:
    # Texto principal
    content = f"Factor ICAM: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos específicos ===
    metadata["metodologia_asociada"] = item.get("metodologia_asociada", "")
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Ejemplos (si existen)
    ejemplos = item.get("ejemplos", [])
    metadata["ejemplos"] = "; ".join([ej["descripcion"] for ej in ejemplos]) if ejemplos else ""

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_icam.append(node)

print(f"✔️ Nodos de Factores ICAM procesados: {len(nodes_icam)}")
if nodes_icam:
    ejemplo = nodes_icam[0]
    print("🔍 Ejemplo de nodo de Factor ICAM:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Factores ICAM procesados: 5
🔍 Ejemplo de nodo de Factor ICAM:
ID: ORG
Texto: Factor ICAM: Factor Organizacional. Factores relacionados con la estructura, cultura, liderazgo o políticas de la organización que contribuyen al incidente.
Metadatos: {'metodologia_asociada': 'ICAM', 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Análisis de causas raíz (ICAM)', 'registro_aplicable_a': 'incidente', 'tags': 'cultura organizacional, gestión, políticas organizacionales, comunicación, presión de producción', 'ejemplos': 'Falta de programa de mantenimiento preventivo o su incumplimiento.; Comunicación deficiente sobre los riesgos laborales.; Presión excesiva para cumplir plazos, comprometiendo la seguridad.; Ausencia de supervisión operacional efectiva o insuficiente cobertura de supervisores.'}


In [24]:
# 📌 Celda 54 - ICAM2 — Inserción segura de Factores ICAM en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_icam = [n.node_id for n in nodes_icam]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_icam,
    where=None,
    limit=999999
)
ids_icam_chroma = set(chroma_check["ids"])
print(f"Factores ICAM ya en Chroma: {len(ids_icam_chroma)}/{len(ids_icam)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_icam_docstore = {id_ for id_ in ids_icam if id_ in ids_docstore}
print(f"Factores ICAM ya en docstore: {len(ids_icam_docstore)}/{len(ids_icam)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_icam if n.node_id not in ids_icam_chroma]
nodos_faltan_docstore = [n for n in nodes_icam if n.node_id not in ids_icam_docstore]

print(f"Nodos de Factores ICAM a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Factores ICAM a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Factores ICAM insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Factores ICAM insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Factores ICAM ya en Chroma: 0/5
Factores ICAM ya en docstore: 0/5
Nodos de Factores ICAM a insertar en Chroma: 5
Nodos de Factores ICAM a insertar en docstore: 5
Factores ICAM insertados en Chroma.
Factores ICAM insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [25]:
# 📌 Celda 55 - ICAM3 — Verificación integral con Factores ICAM incluidos

import re
import random
import pandas as pd
import yaml

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Cargar IDs de Factores ICAM desde el YAML
yaml_icam = os.path.join(ONTOLOGY_DIR, "22_factores_icam_v3.yaml")
with open(yaml_icam, "r", encoding="utf-8") as f:
    factores_yaml = yaml.safe_load(f)
ids_icam_yaml = [item["id"] for item in factores_yaml]
print(f"IDs esperados de Factores ICAM (YAML): {ids_icam_yaml}")

# 3️⃣ Definir patrones por categoría (incluyendo ICAM dinámico)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$",
    "Formatos de Análisis": r"^(ICAM|5WHYS|OBS)$"
}

# 4️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

# Agregar conteo dinámico de Factores ICAM
ids_icam_chroma = [id_ for id_ in ids_chroma if id_ in ids_icam_yaml]
ids_icam_docstore = [id_ for id_ in ids_docstore if id_ in ids_icam_yaml]
resumen.append({
    "Categoría": "Factores ICAM",
    "Chroma": len(ids_icam_chroma),
    "Docstore": len(ids_icam_docstore),
})

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 5️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 6️⃣ Muestreo específico de Factores ICAM
print(f"\nTotal de Factores ICAM en Chroma: {len(ids_icam_chroma)}")
if ids_icam_chroma:
    sample_icam = random.sample(ids_icam_chroma, min(3, len(ids_icam_chroma)))
    print("\nEjemplo específico de nodos de Factores ICAM en Chroma:")
    for sid in sample_icam:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 686
IDs esperados de Factores ICAM (YAML): ['ORG', 'HUM', 'TAR', 'ENT', 'TEC']

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_007
Texto: Factor de Exposición: Contacto con Voltaje Alto. Riesgo de contacto directo o indirecto con circuitos de media o alta tensión durante trabajos de inspección, mantenimiento o conexión eléctrica. Puede  ...
Metadatos: {'areas_asociadas_ids': 'MRA', 'areas_asociadas_nombres': 'Mina Rajo Abierto', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R03', 'riesgos_asociados_nombres': 'Contacto con Energía Eléctrica', 'roles_asociados_ids': 'MRA_ROL_01', 'roles_asociados_nombres': 'Operador de Perforadora', 'tareas_asociadas_ids': 'MRA_TA_003', 'tareas_asociadas_nombres': 'Mantenimiento eléctrico de perforadoras con sistema LOTO', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: TME_TA_002
Texto: Tarea: Intervención en sistemas de media tensión para mantenimiento o pruebas. Intervenir en sistemas de media te

In [26]:
# 📌 Celda 56 - SEV1 — Crear nodos de Niveles de Severidad (enriquecidos con nombres de incidentes)

import yaml
from llama_index.core.schema import TextNode

# === Cargar Tipos de Incidente para mapear IDs → nombres ===
yaml_incidentes = os.path.join(ONTOLOGY_DIR, "25_tipos_incidentes_v1.yaml")
with open(yaml_incidentes, "r", encoding="utf-8") as f:
    tipos_incidentes = yaml.safe_load(f)

mapa_incidentes = {item["id"]: item.get("nombre", "") for item in tipos_incidentes}

# === Cargar catálogo de Niveles de Severidad ===
yaml_severidad = os.path.join(ONTOLOGY_DIR, "27_catalogo_niveles_severidad_v2.yaml")
with open(yaml_severidad, "r", encoding="utf-8") as f:
    catalogo_severidad = yaml.safe_load(f)

nodes_severidad = []
for item in catalogo_severidad:
    # Texto principal
    content = f"Nivel de Severidad: {item.get('nivel', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos específicos ===
    metadata["severidad_agrupada"] = item.get("severidad_agrupada", "")
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # === Enriquecimiento de aplicable_a ===
    ids_aplicables = item.get("aplicable_a", [])
    nombres_aplicables = [mapa_incidentes.get(i, i) for i in ids_aplicables]

    metadata["aplicable_a"] = ", ".join(ids_aplicables)
    metadata["aplicable_a_nombres"] = ", ".join(nombres_aplicables)

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_severidad.append(node)

print(f"✔️ Nodos de Niveles de Severidad procesados: {len(nodes_severidad)}")
if nodes_severidad:
    ejemplo = nodes_severidad[1] if len(nodes_severidad) > 1 else nodes_severidad[0]
    print("🔍 Ejemplo de nodo de Severidad:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Niveles de Severidad procesados: 5
🔍 Ejemplo de nodo de Severidad:
ID: SEV2
Texto: Nivel de Severidad: Moderado. Este nivel describe incidentes con **consecuencias intermedias**. Se caracteriza por **lesiones leves que requieren atención médica básica**, pero que no impiden el retorno al trabajo en el mismo turno o al día siguiente (ej. esguinces leves, cortes superficiales que necesitan sutura simple, irritación ocular). Puede implicar **un corto período de reposo o la reasignación a tareas livianas** sin que ello signifique una pérdida completa de días laborales. En el ámbito material, los daños son **menores pero requieren reparación o reemplazo**, con una **interrupción mínima de la operación** (ej. falla de un componente menor que detiene un equipo por menos de 2 horas, daño a una herramienta especializada).
Metadatos: {'severidad_agrupada': 'BAJA', 'area': 'Todas las Áreas Operacionales', 'aplicabilidad': 'Clasificación de severidad', 'registro_aplicable_a': 'incident

In [27]:
# 📌 Celda 57 - SEV2 — Inserción segura de Niveles de Severidad en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_severidad = [n.node_id for n in nodes_severidad]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_severidad,
    where=None,
    limit=999999
)
ids_severidad_chroma = set(chroma_check["ids"])
print(f"Niveles de Severidad ya en Chroma: {len(ids_severidad_chroma)}/{len(ids_severidad)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_severidad_docstore = {id_ for id_ in ids_severidad if id_ in ids_docstore}
print(f"Niveles de Severidad ya en docstore: {len(ids_severidad_docstore)}/{len(ids_severidad)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_severidad if n.node_id not in ids_severidad_chroma]
nodos_faltan_docstore = [n for n in nodes_severidad if n.node_id not in ids_severidad_docstore]

print(f"Nodos de Severidad a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Severidad a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Niveles de Severidad insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Niveles de Severidad insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Niveles de Severidad ya en Chroma: 0/5
Niveles de Severidad ya en docstore: 0/5
Nodos de Severidad a insertar en Chroma: 5
Nodos de Severidad a insertar en docstore: 5
Niveles de Severidad insertados en Chroma.
Niveles de Severidad insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [28]:
# 📌 Celda 58 - SEV3 — Verificación integral con Niveles de Severidad incluidos

import re
import random
import pandas as pd
import yaml

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Cargar IDs de Severidad desde el YAML
yaml_severidad = os.path.join(ONTOLOGY_DIR, "27_catalogo_niveles_severidad_v2.yaml")
with open(yaml_severidad, "r", encoding="utf-8") as f:
    severidad_yaml = yaml.safe_load(f)
ids_severidad_yaml = [item["id"] for item in severidad_yaml]
print(f"IDs esperados de Severidad (YAML): {ids_severidad_yaml}")

# 3️⃣ Definir patrones por categoría (incluyendo Severidad)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$",
    "Formatos de Análisis": r"^(ICAM|5WHYS|OBS)$",
    "Factores ICAM": r"^(ORG|HUM|TAR|ENT|TEC)$"
}

# 4️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

# Agregar conteo dinámico de Severidad
ids_severidad_chroma = [id_ for id_ in ids_chroma if id_ in ids_severidad_yaml]
ids_severidad_docstore = [id_ for id_ in ids_docstore if id_ in ids_severidad_yaml]
resumen.append({
    "Categoría": "Niveles de Severidad",
    "Chroma": len(ids_severidad_chroma),
    "Docstore": len(ids_severidad_docstore),
})

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 5️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 6️⃣ Muestreo específico de Niveles de Severidad
print(f"\nTotal de Niveles de Severidad en Chroma: {len(ids_severidad_chroma)}")
if ids_severidad_chroma:
    sample_sev = random.sample(ids_severidad_chroma, min(3, len(ids_severidad_chroma)))
    print("\nEjemplo específico de nodos de Niveles de Severidad en Chroma:")
    for sid in sample_sev:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]

        # Mostrar IDs y nombres de aplicable_a
        aplicable_ids = doc_meta.get("aplicable_a", "")
        aplicable_nombres = doc_meta.get("aplicable_a_nombres", "")
        
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Aplicable_a (IDs):", aplicable_ids)
        print("Aplicable_a (Nombres):", aplicable_nombres)
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 691
IDs esperados de Severidad (YAML): ['SEV1', 'SEV2', 'SEV3', 'SEV4', 'SEV5']

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_103
Texto: Factor de Exposición: Trabajo en Altura Cercano a Energía. Realizar inspecciones o mediciones sobre plataformas elevadas cercanas a equipos energizados incrementa el riesgo de caída y arco eléctrico.  ...
Metadatos: {'areas_asociadas_ids': 'TME', 'areas_asociadas_nombres': 'Taller de Mantenimiento Eléctrico', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R01, R03', 'riesgos_asociados_nombres': 'Caída desde Altura, Contacto con Energía Eléctrica', 'roles_asociados_ids': 'TME_ROL_01', 'roles_asociados_nombres': 'Electricista de Media Tensión', 'tareas_asociadas_ids': 'TME_TA_001', 'tareas_asociadas_nombres': 'Inspección visual y termográfica de equipos eléctricos en subestaciones', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: MRA_ROL_05
Texto: Rol: Operador de Motoniveladora/Bulldozer. Responsa

In [30]:
# 📌 Celda 59 - AC1 — Crear nodos de Acciones Correctivas (enriquecidos con riesgos e incidentes/observaciones)
# 📌 Celda 59 - AC1 — Crear nodos de Acciones Correctivas (enriquecidos con riesgos e incidentes/observaciones)

import yaml
from llama_index.core.schema import TextNode

# === Cargar Riesgos ===
yaml_riesgos = os.path.join(ONTOLOGY_DIR, "01_catalogo_riesgos_v8.yaml")
with open(yaml_riesgos, "r", encoding="utf-8") as f:
    riesgos = yaml.safe_load(f)
mapa_riesgos = {item["id"]: item.get("nombre", "") for item in riesgos}

# === Cargar Tipos de Incidente ===
yaml_incidentes = os.path.join(ONTOLOGY_DIR, "25_tipos_incidentes_v1.yaml")
with open(yaml_incidentes, "r", encoding="utf-8") as f:
    tipos_incidentes = yaml.safe_load(f)
mapa_incidentes = {item["id"]: item.get("nombre", "") for item in tipos_incidentes}

# === Cargar Tipos de Observación ===
yaml_observaciones = os.path.join(ONTOLOGY_DIR, "26_tipos_observaciones_v2.yaml")
with open(yaml_observaciones, "r", encoding="utf-8") as f:
    tipos_observaciones = yaml.safe_load(f)
mapa_observaciones = {item["id"]: item.get("nombre", "") for item in tipos_observaciones}

# === Cargar catálogo de Acciones Correctivas ===
yaml_acciones = os.path.join(ONTOLOGY_DIR, "19_catalogo_acciones_correctivas_v1.yaml")
with open(yaml_acciones, "r", encoding="utf-8") as f:
    acciones_correctivas = yaml.safe_load(f)

nodes_acciones = []
for item in acciones_correctivas:
    # Texto principal
    content = f"Acción Correctiva: {item.get('nombre', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Riesgo asociado ===
    riesgo_id = item.get("riesgo_asociado", "")
    metadata["riesgo_asociado_id"] = riesgo_id
    metadata["riesgo_asociado_nombre"] = mapa_riesgos.get(riesgo_id, "Desconocido")

    # === Aplicable a ===
    ids_aplicables = item.get("aplicable_a", [])
    nombres_aplicables = []
    for aid in ids_aplicables:
        if aid in mapa_incidentes:
            nombres_aplicables.append(mapa_incidentes[aid])
        elif aid in mapa_observaciones:
            nombres_aplicables.append(mapa_observaciones[aid])
        else:
            nombres_aplicables.append(aid)

    metadata["aplicable_a_ids"] = ", ".join(ids_aplicables)
    metadata["aplicable_a_nombres"] = ", ".join(nombres_aplicables)

    # === Otros campos ===
    metadata["tipo_control"] = item.get("tipo_control", "")
    metadata["area"] = item.get("area", "")
    metadata["aplicabilidad"] = item.get("aplicabilidad", "")
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # Crear nodo
    node = TextNode(id_=item.get("id"), text=content, metadata=metadata)
    nodes_acciones.append(node)

print(f"✔️ Nodos de Acciones Correctivas procesados: {len(nodes_acciones)}")
if nodes_acciones:
    ejemplo = nodes_acciones[0]
    print("🔍 Ejemplo de nodo de Acción Correctiva:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)



✔️ Nodos de Acciones Correctivas procesados: 40
🔍 Ejemplo de nodo de Acción Correctiva:
ID: AC01
Texto: Acción Correctiva: Actualizar Procedimiento de Trabajo Seguro (PTS) para tareas en altura. Revisar y modificar el Procedimiento de Trabajo Seguro (PTS) existente para actividades en altura. Esto incluye la incorporación de nuevas mejores prácticas, lecciones aprendidas de incidentes, cambios normativos o tecnológicos, y asegurar que el procedimiento sea claro, práctico y fácilmente comprensible por todos los trabajadores involucrados. El objetivo es reforzar las instrucciones para la planificación, ejecución y supervisión segura de los trabajos en altura.
Metadatos: {'riesgo_asociado_id': 'R01', 'riesgo_asociado_nombre': 'Caída desde Altura', 'aplicable_a_ids': 'IMEN, NMS, OPG, OCC', 'aplicable_a_nombres': 'Incidente Menor, Near Miss (Cuasi Accidente), Observación Preventiva General, Observación de Controles Críticos', 'tipo_control': 'preventivo', 'area': 'Todas las Áreas Operaciona

In [31]:
# 📌 Celda 60 - AC2 — Inserción segura de Acciones Correctivas en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_acciones = [n.node_id for n in nodes_acciones]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_acciones,
    where=None,
    limit=999999
)
ids_acciones_chroma = set(chroma_check["ids"])
print(f"Acciones Correctivas ya en Chroma: {len(ids_acciones_chroma)}/{len(ids_acciones)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_acciones_docstore = {id_ for id_ in ids_acciones if id_ in ids_docstore}
print(f"Acciones Correctivas ya en docstore: {len(ids_acciones_docstore)}/{len(ids_acciones)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_acciones if n.node_id not in ids_acciones_chroma]
nodos_faltan_docstore = [n for n in nodes_acciones if n.node_id not in ids_acciones_docstore]

print(f"Nodos de Acciones Correctivas a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos de Acciones Correctivas a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Acciones Correctivas insertadas en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Acciones Correctivas insertadas en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Acciones Correctivas ya en Chroma: 0/40
Acciones Correctivas ya en docstore: 0/40
Nodos de Acciones Correctivas a insertar en Chroma: 40
Nodos de Acciones Correctivas a insertar en docstore: 40
Acciones Correctivas insertadas en Chroma.
Acciones Correctivas insertadas en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [32]:
# 📌 Celda 61 - AC3 — Verificación integral con Acciones Correctivas incluidas

import re
import random
import pandas as pd
import yaml

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Cargar IDs de Acciones desde el YAML
yaml_acciones = os.path.join(ONTOLOGY_DIR, "19_catalogo_acciones_correctivas_v1.yaml")
with open(yaml_acciones, "r", encoding="utf-8") as f:
    acciones_yaml = yaml.safe_load(f)
ids_acciones_yaml = [item["id"] for item in acciones_yaml]
print(f"IDs esperados de Acciones Correctivas (YAML): {ids_acciones_yaml}")

# 3️⃣ Definir patrones por categoría (incluyendo Acciones Correctivas)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$",
    "Formatos de Análisis": r"^(ICAM|5WHYS|OBS)$",
    "Factores ICAM": r"^(ORG|HUM|TAR|ENT|TEC)$",
    "Niveles de Severidad": r"^SEV\d+$"
}

# 4️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

# Agregar conteo dinámico de Acciones Correctivas
ids_acciones_chroma = [id_ for id_ in ids_chroma if id_ in ids_acciones_yaml]
ids_acciones_docstore = [id_ for id_ in ids_docstore if id_ in ids_acciones_yaml]
resumen.append({
    "Categoría": "Acciones Correctivas",
    "Chroma": len(ids_acciones_chroma),
    "Docstore": len(ids_acciones_docstore),
})

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 5️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 6️⃣ Muestreo específico de Acciones Correctivas
print(f"\nTotal de Acciones Correctivas en Chroma: {len(ids_acciones_chroma)}")
if ids_acciones_chroma:
    sample_acc = random.sample(ids_acciones_chroma, min(3, len(ids_acciones_chroma)))
    print("\nEjemplo específico de nodos de Acciones Correctivas en Chroma:")
    for sid in sample_acc:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Riesgo Asociado:", doc_meta.get("riesgo_asociado_id"), "-", doc_meta.get("riesgo_asociado_nombre"))
        print("Aplicable a (IDs):", doc_meta.get("aplicable_a_ids"))
        print("Aplicable a (Nombres):", doc_meta.get("aplicable_a_nombres"))
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 731
IDs esperados de Acciones Correctivas (YAML): ['AC01', 'AC02', 'AC03', 'AC04', 'AC05', 'AC06', 'AC07', 'AC08', 'AC09', 'AC10', 'AC11', 'AC12', 'AC13', 'AC14', 'AC15', 'AC16', 'AC17', 'AC18', 'AC19', 'AC20', 'AC21', 'AC22', 'AC23', 'AC24', 'AC25', 'AC26', 'AC27', 'AC28', 'AC29', 'AC30', 'AC31', 'AC32', 'AC33', 'AC34', 'AC35', 'AC36', 'AC37', 'AC38', 'AC39', 'AC40']

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_008
Texto: Factor de Exposición: Arco Eléctrico. Formación repentina de un arco eléctrico por cortocircuitos, conexiones defectuosas o manipulación inadecuada de interruptores. El fenómeno puede alcanzar tempera ...
Metadatos: {'areas_asociadas_ids': 'MRA', 'areas_asociadas_nombres': 'Mina Rajo Abierto', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R03', 'riesgos_asociados_nombres': 'Contacto con Energía Eléctrica', 'roles_asociados_ids': 'MRA_ROL_01', 'roles_asociados_nombres': 'Operador de Perforadora', 'tareas_asociadas_ids': 'MRA_TA_003', 'tareas_asociadas_nombres': 'Mantenimiento eléctrico de perforadoras con sistema LOTO', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: FE_043
Texto: Factor de Exposición: Visibilidad Reducida por Polvo o Niebla. La presencia de niebla, lluvia intensa o polvo en susp

In [34]:
# 📌 Celda 62 - GLO1 — Crear nodos del Glosario de Seguridad (v5 mejorado con area_ids)

import yaml
from llama_index.core.schema import TextNode

# === Cargar Tipos de Incidente ===
yaml_incidentes = os.path.join(ONTOLOGY_DIR, "25_tipos_incidentes_v1.yaml")
with open(yaml_incidentes, "r", encoding="utf-8") as f:
    tipos_incidentes = yaml.safe_load(f)
mapa_incidentes = {item["id"]: item.get("nombre", "") for item in tipos_incidentes}

# === Cargar Tipos de Observación ===
yaml_observaciones = os.path.join(ONTOLOGY_DIR, "26_tipos_observaciones_v2.yaml")
with open(yaml_observaciones, "r", encoding="utf-8") as f:
    tipos_observaciones = yaml.safe_load(f)
mapa_observaciones = {item["id"]: item.get("nombre", "") for item in tipos_observaciones}

# === Cargar Glosario v5 ===
yaml_glosario = os.path.join(ONTOLOGY_DIR, "28_glosario_seguridad_v5.yaml")
with open(yaml_glosario, "r", encoding="utf-8") as f:
    glosario = yaml.safe_load(f)

nodes_glosario = []
for item in glosario:
    # Texto principal
    content = f"Término de Glosario: {item.get('termino', '')} — {item.get('significado', '')}. {item.get('descripcion', '')}".strip()
    metadata = {}

    # === Campos principales ===
    metadata["categoria"] = item.get("categoria", "")
    metadata["area"] = item.get("area", "")
    metadata["area_ids"] = ", ".join(item.get("area_ids", []))
    metadata["aplicabilidad"] = ", ".join(item.get("aplicabilidad", []))
    metadata["registro_aplicable_a"] = ", ".join(item.get("registro_aplicable_a", []))
    metadata["tags"] = ", ".join(item.get("tags", []))

    # === Enriquecimiento si hay campo aplicable_a ===
    if "aplicable_a" in item:
        ids_aplicables = item.get("aplicable_a", [])
        nombres_aplicables = []
        for aid in ids_aplicables:
            if aid in mapa_incidentes:
                nombres_aplicables.append(mapa_incidentes[aid])
            elif aid in mapa_observaciones:
                nombres_aplicables.append(mapa_observaciones[aid])
            else:
                nombres_aplicables.append(aid)

        metadata["aplicable_a_ids"] = ", ".join(ids_aplicables)
        metadata["aplicable_a_nombres"] = ", ".join(nombres_aplicables)

    # Crear nodo
    node = TextNode(id_=item.get("termino"), text=content, metadata=metadata)
    nodes_glosario.append(node)

print(f"✔️ Nodos de Glosario procesados: {len(nodes_glosario)}")
if nodes_glosario:
    ejemplo = nodes_glosario[0]
    print("🔍 Ejemplo de nodo del Glosario:")
    print("ID:", ejemplo.node_id)
    print("Texto:", ejemplo.text)
    print("Metadatos:", ejemplo.metadata)


✔️ Nodos de Glosario procesados: 13
🔍 Ejemplo de nodo del Glosario:
ID: PTEE
Texto: Término de Glosario: PTEE — Permiso de Trabajo en Equipos Energizados. Documento formal que autoriza la intervención en equipos eléctricos con riesgo de estar energizados, indicando responsables, procedimientos aplicables, riesgos y EPP requeridos.
Metadatos: {'categoria': 'Permisos y Procedimientos', 'area': 'Todas las Áreas Operacionales', 'area_ids': 'MRA, CHP, TME, STM, PLC', 'aplicabilidad': 'Gestión de permisos de trabajo', 'registro_aplicable_a': 'incidente, observacion', 'tags': 'PTEE, permiso, trabajo, equipos, energizados, seguridad eléctrica'}


In [35]:
# 📌 Celda 63 - GLO2 — Inserción segura del Glosario en docstore + Chroma

from llama_index.core import VectorStoreIndex
import json
import os

# 1️⃣ Extraer IDs
ids_glosario = [n.node_id for n in nodes_glosario]

# 2️⃣ Consultar en Chroma
chroma_check = storage_context.vector_store._collection.get(
    ids=ids_glosario,
    where=None,
    limit=999999
)
ids_glosario_chroma = set(chroma_check["ids"])
print(f"Términos del Glosario ya en Chroma: {len(ids_glosario_chroma)}/{len(ids_glosario)}")

# 3️⃣ Consultar en docstore
ids_docstore = set(storage_context.docstore.docs.keys())
ids_glosario_docstore = {id_ for id_ in ids_glosario if id_ in ids_docstore}
print(f"Términos del Glosario ya en docstore: {len(ids_glosario_docstore)}/{len(ids_glosario)}")

# 4️⃣ Filtrar faltantes
nodos_faltan_chroma = [n for n in nodes_glosario if n.node_id not in ids_glosario_chroma]
nodos_faltan_docstore = [n for n in nodes_glosario if n.node_id not in ids_glosario_docstore]

print(f"Nodos del Glosario a insertar en Chroma: {len(nodos_faltan_chroma)}")
print(f"Nodos del Glosario a insertar en docstore: {len(nodos_faltan_docstore)}")

# 5️⃣ Insertar en Chroma
if nodos_faltan_chroma:
    _ = VectorStoreIndex(
        nodos_faltan_chroma,
        storage_context=storage_context,
        embed_model=embed_model,
        store_nodes_in_index=True
    )
    print("Términos del Glosario insertados en Chroma.")

# 6️⃣ Insertar en docstore
if nodos_faltan_docstore:
    for node in nodos_faltan_docstore:
        storage_context.docstore.add_documents([node])
    print("Términos del Glosario insertados en docstore.")

# 7️⃣ Persistir cambios
storage_context.persist(persist_dir=STORAGE_DIR)

# 8️⃣ Guardar docstore.json como respaldo
docstore_path = os.path.join(STORAGE_DIR, "docstore.json")
with open(docstore_path, "w", encoding="utf-8") as f:
    json.dump(storage_context.docstore.to_dict(), f, ensure_ascii=False, indent=2)

print(f"✅ Persistencia completada. docstore.json actualizado en: {docstore_path}")


Términos del Glosario ya en Chroma: 9/13
Términos del Glosario ya en docstore: 9/13
Nodos del Glosario a insertar en Chroma: 4
Nodos del Glosario a insertar en docstore: 4
Términos del Glosario insertados en Chroma.
Términos del Glosario insertados en docstore.
✅ Persistencia completada. docstore.json actualizado en: C:\batch001\store_test_yaml2\docstore.json


In [36]:
# 📌 Celda 64 - GLO3 — Verificación integral con Glosario incluido

import re
import random
import pandas as pd
import yaml

# 1️⃣ Obtener registros actuales en Chroma
chroma_data = storage_context.vector_store._collection.get(
    ids=None,
    limit=999999,
    include=["metadatas", "documents"]
)
ids_chroma = set(chroma_data["ids"])
print(f"Total de IDs en Chroma: {len(ids_chroma)}")

# 2️⃣ Cargar IDs del Glosario desde el YAML
yaml_glosario = os.path.join(ONTOLOGY_DIR, "28_glosario_seguridad_v5.yaml")
with open(yaml_glosario, "r", encoding="utf-8") as f:
    glosario_yaml = yaml.safe_load(f)
ids_glosario_yaml = [item["termino"] for item in glosario_yaml]
print(f"IDs esperados del Glosario (YAML): {ids_glosario_yaml[:10]}... total={len(ids_glosario_yaml)}")

# 3️⃣ Definir patrones por categoría (incluyendo Glosario)
categorias_regex = {
    "Riesgos": r"^R\d+$",
    "Controles": r"^R\d+(CP|CM|CC)\d+$",
    "Causas": r"^R\d+CA\d+$",
    "Consecuencias": r"^R\d+CON\d+$",
    "Factores de Degradación": r"^R\d+FD\d+$",
    "Barreras de Recuperación": r"^R\d+BR\d+$",
    "Factores de Exposición": r"^FE_\d+$",
    "Áreas Operacionales": r"^(MRA|CHP|TME|STM|PLC)$",
    "Roles": r"^[A-Z]{3}_ROL_\d+$",
    "Tareas": r"^[A-Z]{3}_TA_\d+$",
    "EPP": r"^EPP_\d+$",
    "Peligros": r"^P\d+$",
    "Eventos": r"^EVT\d+$",
    "Tipos de Incidente": r"^(IMAY|IMEN|NMIS|HZRD)$",
    "Tipos de Observación": r"^(OPG|OCC)$",
    "Formatos de Análisis": r"^(ICAM|5WHYS|OBS)$",
    "Factores ICAM": r"^(ORG|HUM|TAR|ENT|TEC)$",
    "Niveles de Severidad": r"^SEV\d+$"
}

# 4️⃣ Conteo por categoría
ids_docstore = set(storage_context.docstore.docs.keys())
resumen = []
for nombre, patron in categorias_regex.items():
    ids_cat_chroma = [id_ for id_ in ids_chroma if re.match(patron, id_)]
    ids_cat_docstore = [id_ for id_ in ids_docstore if re.match(patron, id_)]
    resumen.append({
        "Categoría": nombre,
        "Chroma": len(ids_cat_chroma),
        "Docstore": len(ids_cat_docstore),
    })

# Agregar conteo dinámico de Glosario
ids_glosario_chroma = [id_ for id_ in ids_chroma if id_ in ids_glosario_yaml]
ids_glosario_docstore = [id_ for id_ in ids_docstore if id_ in ids_glosario_yaml]
resumen.append({
    "Categoría": "Glosario",
    "Chroma": len(ids_glosario_chroma),
    "Docstore": len(ids_glosario_docstore),
})

df_resumen = pd.DataFrame(resumen)
print("\nResumen de nodos por categoría:")
display(df_resumen)

# 5️⃣ Muestreo aleatorio global
if ids_chroma:
    sample_ids = random.sample(list(ids_chroma), min(3, len(ids_chroma)))
    print("\nEjemplo aleatorio de nodos en Chroma (global):")
    for sid in sample_ids:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})

# 6️⃣ Muestreo específico del Glosario
print(f"\nTotal de términos del Glosario en Chroma: {len(ids_glosario_chroma)}")
if ids_glosario_chroma:
    sample_glosario = random.sample(ids_glosario_chroma, min(3, len(ids_glosario_chroma)))
    print("\nEjemplo específico de nodos del Glosario en Chroma:")
    for sid in sample_glosario:
        idx = chroma_data["ids"].index(sid)
        doc_text = chroma_data["documents"][idx]
        doc_meta = chroma_data["metadatas"][idx]
        print(f"\nID: {sid}")
        print("Texto:", doc_text[:200], "..." if len(doc_text) > 200 else "")
        print("Área IDs:", doc_meta.get("area_ids", ""))
        if "aplicable_a_ids" in doc_meta:
            print("Aplicable_a (IDs):", doc_meta.get("aplicable_a_ids"))
            print("Aplicable_a (Nombres):", doc_meta.get("aplicable_a_nombres"))
        print("Metadatos:", {k: v for k, v in doc_meta.items() if not k.startswith('_')})


Total de IDs en Chroma: 735
IDs esperados del Glosario (YAML): ['PTEE', 'PTS', 'SPDC', 'LOTO', 'ICAM', '5WHYS', 'OBS', 'OCC', 'OPG', 'IMAY']... total=13

Resumen de nodos por categoría:


,Categoría,Chroma,Docstore
0,Riesgos,3,3
1,Controles,48,48
2,Causas,18,18
3,Consecuencias,18,18
4,Factores de Degradación,12,12
5,Barreras de Recuperación,11,11
6,Factores de Exposición,211,211
7,Áreas Operacionales,5,5
8,Roles,45,45
9,Tareas,96,96



Ejemplo aleatorio de nodos en Chroma (global):

ID: FE_189
Texto: Factor de Exposición: Contacto Directo con Equipos Energizados. Manipular cables, bornes o equipos bajo tensión sin verificación ni bloqueo puede provocar descarga eléctrica o quemaduras por arco. 
Metadatos: {'areas_asociadas_ids': 'PLC', 'areas_asociadas_nombres': 'Planta Concentradora', 'doc_id': 'None', 'document_id': 'None', 'ref_doc_id': 'None', 'riesgos_asociados_ids': 'R03', 'riesgos_asociados_nombres': 'Contacto con Energía Eléctrica', 'roles_asociados_ids': 'PLC_ROL_05', 'roles_asociados_nombres': 'Mecánico de Planta Concentradora', 'tareas_asociadas_ids': 'PLC_TA_009', 'tareas_asociadas_nombres': 'Diagnóstico y mantenimiento de motores eléctricos industriales', 'tiene_areas_asociadas': True, 'tiene_riesgos_asociados': True, 'tiene_roles_asociados': True, 'tiene_tareas_asociadas': True}

ID: FE_135
Texto: Factor de Exposición: Trabajo en Posiciones Forzadas en Tableros. Las tareas de configuración y ajuste den